In [1]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from general_functions import *
from spherical_flux import *
%load_ext autoreload
%autoreload
%autoreload 2

plt.style.use(['seaborn-poster'])

f = h5py.File('batsrus_3d_multi_fluid_95000_10k_3deg.h5','r')

dat_x_values = f['x'][:]
dat_y_values = f['y'][:]
dat_z_values = f['z'][:]

def conversion_sphere_to_cart(r,theta,phi):
    '''converts spherical coords to x,y,z
    INPUTS:
    r: in Rm
    theta: (rad)
    phi: (rad)'''

    x = r * np.cos(theta) * np.sin(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(phi)
    return x, y, z

def conversion_cart_to_sphere(x,y,z):
    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arctan2(y,x)
    phi = np.arccos(z/r)
    return r,theta,phi

dat_r_values, dat_theta_values, dat_phi_values = conversion_cart_to_sphere(dat_x_values, dat_y_values, dat_z_values)

# 100km:

In [2]:
r_sphere_100km = (3390.0+100.0) / 3390.0 #300km above surface
one_hundred_thirty_km = (130.0) / 3390.0
r = r_sphere_100km
lon = np.arange(-90,271,1)
lat = np.arange(-90,91,1)

phi = -1*(lat-90)*np.pi/180.0
theta = lon*np.pi/180.0

phi_v, theta_v = np.meshgrid(phi, theta)

#Make face centers
phi_f = 0.5*(phi_v[1:,1:]+phi_v[:-1,:-1])
theta_f = 0.5*(theta_v[1:,1:]+theta_v[:-1,:-1])
lat_f = -1*phi_f*180/np.pi+90
lon_f = theta_f*180/np.pi

x = (r*np.cos(theta_f)*np.sin(phi_f)).flatten()
y = (r*np.sin(theta_f)*np.sin(phi_f)).flatten()    
z = (r*np.cos(phi_f)).flatten()
interpolation_spheres_coords_cartesian = np.array([x,y,z])
#p = np.array([x, y, z])
#normal = p/np.sqrt(np.sum(p**2, axis=0))

In [3]:
number_of_points_in_sphere = np.array([])
interpolated_values_array = np.array([])

for i in range(len(interpolation_spheres_coords_cartesian[0])):
    interp_r = np.sqrt(np.sum(interpolation_spheres_coords_cartesian[:,i]**2))
    r_dist = np.abs(dat_r_values - interp_r)
    distance = ((dat_x_values - interpolation_spheres_coords_cartesian[0,i])**2 +
                (dat_y_values - interpolation_spheres_coords_cartesian[1,i])**2 +
                (dat_z_values - interpolation_spheres_coords_cartesian[2,i])**2)
    
    good_points = np.logical_and(distance<=(one_hundred_thirty_km**2),
                                r_dist<=5.0/3390.0)
    good_x, good_y, good_z, good_distances = dat_x_values[good_points], dat_y_values[good_points], dat_z_values[good_points], distance[good_points]
    number_of_points_in_sphere = np.append(number_of_points_in_sphere, len(good_distances))

    dat_O2_p1_num_dens_values = f['O2_p1_number_density'][:][good_points] #Pulling out values at new indices within interp sphere
    dat_O2_p1_v_x = f['O2_p1_velocity_x'][:][good_points]
    dat_O2_p1_v_y = f['O2_p1_velocity_y'][:][good_points]
    dat_O2_p1_v_z = f['O2_p1_velocity_z'][:][good_points]
    dat_O2_p1_v_tot = np.sqrt(dat_O2_p1_v_x**2 + dat_O2_p1_v_y**2 + dat_O2_p1_v_z**2)

    dat_O_p1_num_dens_values = f['O_p1_number_density'][:][good_points]
    dat_O_p1_v_x = f['O_p1_velocity_x'][:][good_points]
    dat_O_p1_v_y = f['O_p1_velocity_y'][:][good_points]
    dat_O_p1_v_z = f['O_p1_velocity_z'][:][good_points]
    dat_O_p1_v_tot = np.sqrt(dat_O_p1_v_x**2 + dat_O_p1_v_y**2 + dat_O_p1_v_z**2)
    
    mag_field_x = f['magnetic_field_x'][:][good_points]
    mag_field_y = f['magnetic_field_y'][:][good_points]
    mag_field_z = f['magnetic_field_z'][:][good_points]
    mag_field_tot = np.sqrt(mag_field_x**2 + mag_field_y**2 + mag_field_z**2)
    
    p = np.array([good_x, good_y, good_z])
    normal = p/np.sqrt(np.sum(p**2, axis=0))
    norm = np.array([mag_field_x,mag_field_y,mag_field_z])
    mag_field_norm = np.sum(normal*norm, axis=0)
    
    #J = np.array([f['current_x'][:][good_points], f['current_y'][:][good_points], f['current_z'][:][good_points]])
    #B = np.array([mag_field_x, mag_field_y, mag_field_z])
    #J_cross_B_x = J[1]*B[2] - J[2]*B[1]
    #J_cross_B_y = J[2]*B[0] - J[0]*B[2]
    #J_cross_B_z = J[0]*B[1] - J[1]*B[0]
    #J_cross_B_tot = np.sqrt(J_cross_B_x**2 + J_cross_B_y**2 + J_cross_B_z**2)
    
    interpolation_fields = np.array([dat_O2_p1_num_dens_values, dat_O2_p1_v_x, dat_O2_p1_v_y, dat_O2_p1_v_z, dat_O2_p1_v_tot,
                             dat_O_p1_num_dens_values, dat_O_p1_v_x, dat_O_p1_v_y, dat_O_p1_v_z, dat_O_p1_v_tot, 
                             mag_field_x, mag_field_y, mag_field_z, mag_field_tot, mag_field_norm])#J_cross_B_x,J_cross_B_y,J_cross_B_z,J_cross_B_tot

    distances = good_distances #array of distances from center of sphere to points
    weights = 1.0 / (distances) #1/di factor

    for n in range(len(interpolation_fields)):
        numerator = np.sum(weights*interpolation_fields[n])
        denominator = np.sum(weights)
        interpolated_values_array = np.append(interpolated_values_array, (numerator / denominator))

reshaped_interpolated_values_array = np.reshape(interpolated_values_array, (len(interpolation_spheres_coords_cartesian[0]),15))

In [4]:
print np.min(number_of_points_in_sphere)
print np.max(number_of_points_in_sphere)

1.0
88.0


In [10]:
print reshaped_interpolated_values_array[0]

[ 1.39186469e+03  1.50499651e-03  2.22619117e-04 -6.03104099e-04
  1.63659211e-03  4.80234870e-01  2.13969065e-03  3.06655952e-04
  1.82502384e-03  2.82896444e-03  3.98223399e+01  1.30416588e+02
  5.61764086e+02  5.85693518e+02 -5.61764086e+02]


In [11]:
plt.style.use(['seaborn-poster'])
interpolation_sphere_coords_spherical = conversion_cart_to_sphere(interpolation_spheres_coords_cartesian[0], interpolation_spheres_coords_cartesian[1], interpolation_spheres_coords_cartesian[2])
lat_f_array = -1* interpolation_sphere_coords_spherical[2] *180/np.pi+90
lon_f_array = interpolation_sphere_coords_spherical[1] *180/np.pi

rounded_lat_f_array = np.round(lat_f_array, 1)
rounded_lon_f_array = np.round(lon_f_array, 1)

O2_num_dens_data = reshaped_interpolated_values_array[:,0]
O2_vtot_data = reshaped_interpolated_values_array[:,4]

#num_dens_0_5 = O2_num_dens_data[np.argwhere(rounded_lon_f_array==0.5)]
#lat_values = rounded_lat_f_array[np.argwhere(rounded_lon_f_array==0.5)]
#plt.plot(num_dens_0_5,lat_values, label='lon 0.5', lw=2)

In [22]:
stack_coords = np.column_stack(interpolation_sphere_coords_spherical)
print stack_coords[0]

[ 1.02949853 -1.56206968  3.13286601]


In [28]:
stacked_interp_vals = np.hstack((reshaped_interpolated_values_array, stack_coords))
print stacked_interp_vals

[[ 1.39186469e+03  1.50499651e-03  2.22619117e-04 ...  1.02949853e+00
  -1.56206968e+00  3.13286601e+00]
 [ 1.39383618e+03  1.50525759e-03  2.23018467e-04 ...  1.02949853e+00
  -1.56206968e+00  3.11541271e+00]
 [ 1.39637889e+03  1.50886845e-03  2.30411104e-04 ...  1.02949853e+00
  -1.56206968e+00  3.09795942e+00]
 ...
 [ 6.13889093e+03 -5.29693459e-05  3.18465184e-06 ...  1.02949853e+00
  -1.57952297e+00  4.36332313e-02]
 [ 6.17138431e+03 -5.29559742e-05  8.54274938e-07 ...  1.02949853e+00
  -1.57952297e+00  2.61799388e-02]
 [ 6.18820081e+03 -5.27464518e-05  4.59078426e-08 ...  1.02949853e+00
  -1.57952297e+00  8.72664626e-03]]


In [34]:
#SAVING values to txt file
file = open('LR_100km_interp_vals.txt','ab') #opens file to append without overwriting
#dataout = (reshaped_interpolated_values_array, interpolation_sphere_coords_spherical))
np.savetxt(file, stacked_interp_vals, delimiter=',',header=' O2+_num_dens, O2+_vx, O2+_vy, O2+_vz, O2+_vtot, O+_num_dens, O+_vx, O+_vy, O+_vz, O+_vtot, mag_x, mag_y, mag_z, mag_tot, mag_norm, r, theta, phi',
           fmt='%8.10e',comments='#') #saves columnated values as FWHM,xmin,xmax
file.close()

## V cross B values:

In [ ]:
number_of_points_in_sphere = np.array([])
interpolated_values_array = np.array([])

for i in range(len(interpolation_spheres_coords_cartesian[0])):
    interp_r = np.sqrt(np.sum(interpolation_spheres_coords_cartesian[:,i]**2))
    r_dist = np.abs(dat_r_values - interp_r)
    distance = ((dat_x_values - interpolation_spheres_coords_cartesian[0,i])**2 +
                (dat_y_values - interpolation_spheres_coords_cartesian[1,i])**2 +
                (dat_z_values - interpolation_spheres_coords_cartesian[2,i])**2)
    
    good_points = np.logical_and(distance<=(one_hundred_thirty_km**2),
                                r_dist<=5.0/3390.0)
    good_x, good_y, good_z, good_distances = dat_x_values[good_points], dat_y_values[good_points], dat_z_values[good_points], distance[good_points]
    number_of_points_in_sphere = np.append(number_of_points_in_sphere, len(good_distances))

    dat_H_p1_num_dens_values = f['H_p1_number_density'][:][good_points] #Pulling out values at new indices within interp sphere
    dat_H_p1_v_x = f['H_p1_velocity_x'][:][good_points]
    dat_H_p1_v_y = f['H_p1_velocity_y'][:][good_points]
    dat_H_p1_v_z = f['H_p1_velocity_z'][:][good_points]
    dat_H_p1_v_tot = np.sqrt(dat_H_p1_v_x**2 + dat_H_p1_v_y**2 + dat_H_p1_v_z**2)

    dat_velocity_x = f['velocity_x'][:][good_points] #background fluid velocities
    dat_velocity_y = f['velocity_y'][:][good_points]
    dat_velocity_z = f['velocity_z'][:][good_points]
    dat_velocity_total = np.sqrt(dat_velocity_x**2 + dat_velocity_y**2 + dat_velocity_z**2)
    
    dat_current_x = f['current_x'][:][good_points]
    dat_current_y = f['current_y'][:][good_points]
    dat_current_z = f['current_z'][:][good_points]
    dat_current_total = np.sqrt(dat_current_x**2 + dat_current_y**2 + dat_current_z**2)
    
    #J = np.array([f['current_x'][:][good_points], f['current_y'][:][good_points], f['current_z'][:][good_points]])
    #B = np.array([mag_field_x, mag_field_y, mag_field_z])
    #J_cross_B_x = J[1]*B[2] - J[2]*B[1]
    #J_cross_B_y = J[2]*B[0] - J[0]*B[2]
    #J_cross_B_z = J[0]*B[1] - J[1]*B[0]
    #J_cross_B_tot = np.sqrt(J_cross_B_x**2 + J_cross_B_y**2 + J_cross_B_z**2)
    
    interpolation_fields = np.array([dat_H_p1_num_dens_values, dat_H_p1_v_x, dat_H_p1_v_y, dat_H_p1_v_z, dat_H_p1_v_tot,
                                    dat_velocity_x, dat_velocity_y, dat_velocity_z, dat_velocity_total, dat_current_x,
                                    dat_current_y, dat_current_z, dat_current_total])#J_cross_B_x,J_cross_B_y,J_cross_B_z,J_cross_B_tot

    distances = good_distances #array of distances from center of sphere to points
    weights = 1.0 / (distances) #1/di factor

    for n in range(len(interpolation_fields)):
        numerator = np.sum(weights*interpolation_fields[n])
        denominator = np.sum(weights)
        interpolated_values_array = np.append(interpolated_values_array, (numerator / denominator))

reshaped_interpolated_values_array = np.reshape(interpolated_values_array, (len(interpolation_spheres_coords_cartesian[0]),13))

In [ ]:
#SAVING values to txt file
file = open('LR_100km_interp_vals_v_cross_b.txt','ab') #opens file to append without overwriting
#dataout = (reshaped_interpolated_values_array, interpolation_sphere_coords_spherical))
np.savetxt(file, reshaped_interpolated_values_array, delimiter=',',header=' H+_num_dens, H+_vx, H+_vy, H+_vz, H+_vtot, velocity_x, velocity_y, velocity_z, velocity_total, current_x, current_y, current_z, current_total',
           fmt='%8.10e',comments='#') #saves columnated values as FWHM,xmin,xmax
file.close()

# 200km:

In [3]:
r_sphere_200km = (3390.0+200.0) / 3390.0 #300km above surface
one_hundred_thirty_five_km = (135.0) / 3390.0
r = r_sphere_200km
lon = np.arange(-90,271,1)
lat = np.arange(-90,91,1)

phi = -1*(lat-90)*np.pi/180.0
theta = lon*np.pi/180.0

phi_v, theta_v = np.meshgrid(phi, theta)

#Make face centers
phi_f = 0.5*(phi_v[1:,1:]+phi_v[:-1,:-1])
theta_f = 0.5*(theta_v[1:,1:]+theta_v[:-1,:-1])
lat_f = -1*phi_f*180/np.pi+90
lon_f = theta_f*180/np.pi

x = (r*np.cos(theta_f)*np.sin(phi_f)).flatten()
y = (r*np.sin(theta_f)*np.sin(phi_f)).flatten()    
z = (r*np.cos(phi_f)).flatten()
interpolation_spheres_coords_cartesian = np.array([x,y,z])

In [4]:
number_of_points_in_sphere = np.array([])
interpolated_values_array = np.array([])

for i in range(len(interpolation_spheres_coords_cartesian[0])):
    interp_r = np.sqrt(np.sum(interpolation_spheres_coords_cartesian[:,i]**2))
    r_dist = np.abs(dat_r_values - interp_r)
    distance = ((dat_x_values - interpolation_spheres_coords_cartesian[0,i])**2 +
                (dat_y_values - interpolation_spheres_coords_cartesian[1,i])**2 +
                (dat_z_values - interpolation_spheres_coords_cartesian[2,i])**2)
    
    good_points = np.logical_and(distance<=(one_hundred_thirty_five_km**2),
                                r_dist<=25.0/3390.0)
    good_x, good_y, good_z, good_distances = dat_x_values[good_points], dat_y_values[good_points], dat_z_values[good_points], distance[good_points]
    number_of_points_in_sphere = np.append(number_of_points_in_sphere, len(good_distances))

    dat_O2_p1_num_dens_values = f['O2_p1_number_density'][:][good_points] #Pulling out values at new indices within interp sphere
    dat_O2_p1_v_x = f['O2_p1_velocity_x'][:][good_points]
    dat_O2_p1_v_y = f['O2_p1_velocity_y'][:][good_points]
    dat_O2_p1_v_z = f['O2_p1_velocity_z'][:][good_points]
    dat_O2_p1_v_tot = np.sqrt(dat_O2_p1_v_x**2 + dat_O2_p1_v_y**2 + dat_O2_p1_v_z**2)

    dat_O_p1_num_dens_values = f['O_p1_number_density'][:][good_points]
    dat_O_p1_v_x = f['O_p1_velocity_x'][:][good_points]
    dat_O_p1_v_y = f['O_p1_velocity_y'][:][good_points]
    dat_O_p1_v_z = f['O_p1_velocity_z'][:][good_points]
    dat_O_p1_v_tot = np.sqrt(dat_O_p1_v_x**2 + dat_O_p1_v_y**2 + dat_O_p1_v_z**2)
    
    mag_field_x = f['magnetic_field_x'][:][good_points]
    mag_field_y = f['magnetic_field_y'][:][good_points]
    mag_field_z = f['magnetic_field_z'][:][good_points]
    mag_field_tot = np.sqrt(mag_field_x**2 + mag_field_y**2 + mag_field_z**2)
    
    p = np.array([good_x, good_y, good_z])
    normal = p/np.sqrt(np.sum(p**2, axis=0))
    norm = np.array([mag_field_x,mag_field_y,mag_field_z])
    mag_field_norm = np.sum(normal*norm, axis=0)
    
    interpolation_fields = np.array([dat_O2_p1_num_dens_values, dat_O2_p1_v_x, dat_O2_p1_v_y, dat_O2_p1_v_z, dat_O2_p1_v_tot,
                             dat_O_p1_num_dens_values, dat_O_p1_v_x, dat_O_p1_v_y, dat_O_p1_v_z, dat_O_p1_v_tot, 
                             mag_field_x, mag_field_y, mag_field_z, mag_field_tot, mag_field_norm])

    distances = good_distances #array of distances from center of sphere to points
    weights = 1.0 / (distances) #1/di factor

    for n in range(len(interpolation_fields)):
        numerator = np.sum(weights*interpolation_fields[n])
        denominator = np.sum(weights)
        interpolated_values_array = np.append(interpolated_values_array, (numerator / denominator))

reshaped_interpolated_values_array = np.reshape(interpolated_values_array, (len(interpolation_spheres_coords_cartesian[0]),15))

In [5]:
print np.min(number_of_points_in_sphere)
print np.max(number_of_points_in_sphere)

5.0
441.0


In [6]:
interpolation_sphere_coords_spherical = conversion_cart_to_sphere(interpolation_spheres_coords_cartesian[0], interpolation_spheres_coords_cartesian[1], interpolation_spheres_coords_cartesian[2])
stack_coords = np.column_stack(interpolation_sphere_coords_spherical)
stacked_interp_vals = np.hstack((reshaped_interpolated_values_array, stack_coords))

#SAVING values to txt file
file = open('LR_200km_interp_vals.txt','ab') #opens file to append without overwriting
#dataout = (reshaped_interpolated_values_array, interpolation_sphere_coords_spherical))
np.savetxt(file, stacked_interp_vals, delimiter=',',header=' O2+_num_dens, O2+_vx, O2+_vy, O2+_vz, O2+_vtot, O+_num_dens, O+_vx, O+_vy, O+_vz, O+_vtot, mag_x, mag_y, mag_z, mag_tot, mag_norm, r, theta, phi',
           fmt='%8.10e',comments='#') #saves columnated values as FWHM,xmin,xmax
file.close()

## V cross B forces:

In [ ]:
r_sphere_200km = (3390.0+200.0) / 3390.0 #300km above surface
one_hundred_thirty_five_km = (135.0) / 3390.0
r = r_sphere_200km
lon = np.arange(-90,271,1)
lat = np.arange(-90,91,1)

phi = -1*(lat-90)*np.pi/180.0
theta = lon*np.pi/180.0

phi_v, theta_v = np.meshgrid(phi, theta)

#Make face centers
phi_f = 0.5*(phi_v[1:,1:]+phi_v[:-1,:-1])
theta_f = 0.5*(theta_v[1:,1:]+theta_v[:-1,:-1])
lat_f = -1*phi_f*180/np.pi+90
lon_f = theta_f*180/np.pi

x = (r*np.cos(theta_f)*np.sin(phi_f)).flatten()
y = (r*np.sin(theta_f)*np.sin(phi_f)).flatten()    
z = (r*np.cos(phi_f)).flatten()
interpolation_spheres_coords_cartesian = np.array([x,y,z])

number_of_points_in_sphere = np.array([])
interpolated_values_array = np.array([])

for i in range(len(interpolation_spheres_coords_cartesian[0])):
    interp_r = np.sqrt(np.sum(interpolation_spheres_coords_cartesian[:,i]**2))
    r_dist = np.abs(dat_r_values - interp_r)
    distance = ((dat_x_values - interpolation_spheres_coords_cartesian[0,i])**2 +
                (dat_y_values - interpolation_spheres_coords_cartesian[1,i])**2 +
                (dat_z_values - interpolation_spheres_coords_cartesian[2,i])**2)
    
    good_points = np.logical_and(distance<=(one_hundred_thirty_five_km**2),
                                r_dist<=25.0/3390.0)
    good_x, good_y, good_z, good_distances = dat_x_values[good_points], dat_y_values[good_points], dat_z_values[good_points], distance[good_points]
    number_of_points_in_sphere = np.append(number_of_points_in_sphere, len(good_distances))

    dat_H_p1_num_dens_values = f['H_p1_number_density'][:][good_points] #Pulling out values at new indices within interp sphere
    dat_H_p1_v_x = f['H_p1_velocity_x'][:][good_points]
    dat_H_p1_v_y = f['H_p1_velocity_y'][:][good_points]
    dat_H_p1_v_z = f['H_p1_velocity_z'][:][good_points]
    dat_H_p1_v_tot = np.sqrt(dat_H_p1_v_x**2 + dat_H_p1_v_y**2 + dat_H_p1_v_z**2)

    dat_velocity_x = f['velocity_x'][:][good_points] #background fluid velocities
    dat_velocity_y = f['velocity_y'][:][good_points]
    dat_velocity_z = f['velocity_z'][:][good_points]
    dat_velocity_total = np.sqrt(dat_velocity_x**2 + dat_velocity_y**2 + dat_velocity_z**2)
    
    dat_current_x = f['current_x'][:][good_points]
    dat_current_y = f['current_y'][:][good_points]
    dat_current_z = f['current_z'][:][good_points]
    dat_current_total = np.sqrt(dat_current_x**2 + dat_current_y**2 + dat_current_z**2)
    
    #J = np.array([f['current_x'][:][good_points], f['current_y'][:][good_points], f['current_z'][:][good_points]])
    #B = np.array([mag_field_x, mag_field_y, mag_field_z])
    #J_cross_B_x = J[1]*B[2] - J[2]*B[1]
    #J_cross_B_y = J[2]*B[0] - J[0]*B[2]
    #J_cross_B_z = J[0]*B[1] - J[1]*B[0]
    #J_cross_B_tot = np.sqrt(J_cross_B_x**2 + J_cross_B_y**2 + J_cross_B_z**2)
    
    interpolation_fields = np.array([dat_H_p1_num_dens_values, dat_H_p1_v_x, dat_H_p1_v_y, dat_H_p1_v_z, dat_H_p1_v_tot,
                                    dat_velocity_x, dat_velocity_y, dat_velocity_z, dat_velocity_total, dat_current_x,
                                    dat_current_y, dat_current_z, dat_current_total])#J_cross_B_x,J_cross_B_y,J_cross_B_z,J_cross_B_tot

    distances = good_distances #array of distances from center of sphere to points
    weights = 1.0 / (distances) #1/di factor

    for n in range(len(interpolation_fields)):
        numerator = np.sum(weights*interpolation_fields[n])
        denominator = np.sum(weights)
        interpolated_values_array = np.append(interpolated_values_array, (numerator / denominator))

reshaped_interpolated_values_array = np.reshape(interpolated_values_array, (len(interpolation_spheres_coords_cartesian[0]),13))

In [ ]:
#SAVING values to txt file
file = open('LR_200km_interp_vals_v_cross_b.txt','ab') #opens file to append without overwriting
#dataout = (reshaped_interpolated_values_array, interpolation_sphere_coords_spherical))
np.savetxt(file, reshaped_interpolated_values_array, delimiter=',',header=' H+_num_dens, H+_vx, H+_vy, H+_vz, H+_vtot, velocity_x, velocity_y, velocity_z, velocity_total, current_x, current_y, current_z, current_total',
           fmt='%8.10e',comments='#') #saves columnated values as FWHM,xmin,xmax
file.close()

# 300km:

In [7]:
r_sphere_300km = (3390.0+300.0) / 3390.0 #300km above surface
one_hundred_forty_five_km = (145.0) / 3390.0
r = r_sphere_300km
lon = np.arange(-90,271,1)
lat = np.arange(-90,91,1)

phi = -1*(lat-90)*np.pi/180.0
theta = lon*np.pi/180.0

phi_v, theta_v = np.meshgrid(phi, theta)

#Make face centers
phi_f = 0.5*(phi_v[1:,1:]+phi_v[:-1,:-1])
theta_f = 0.5*(theta_v[1:,1:]+theta_v[:-1,:-1])
lat_f = -1*phi_f*180/np.pi+90
lon_f = theta_f*180/np.pi

x = (r*np.cos(theta_f)*np.sin(phi_f)).flatten()
y = (r*np.sin(theta_f)*np.sin(phi_f)).flatten()    
z = (r*np.cos(phi_f)).flatten()
interpolation_spheres_coords_cartesian = np.array([x,y,z])

In [8]:
number_of_points_in_sphere = np.array([])
interpolated_values_array = np.array([])

for i in range(len(interpolation_spheres_coords_cartesian[0])):
    interp_r = np.sqrt(np.sum(interpolation_spheres_coords_cartesian[:,i]**2))
    r_dist = np.abs(dat_r_values - interp_r)
    distance = ((dat_x_values - interpolation_spheres_coords_cartesian[0,i])**2 +
                (dat_y_values - interpolation_spheres_coords_cartesian[1,i])**2 +
                (dat_z_values - interpolation_spheres_coords_cartesian[2,i])**2)
    
    good_points = np.logical_and(distance<=(one_hundred_forty_five_km**2),
                                r_dist<=15.0/3390.0)
    good_x, good_y, good_z, good_distances = dat_x_values[good_points], dat_y_values[good_points], dat_z_values[good_points], distance[good_points]
    number_of_points_in_sphere = np.append(number_of_points_in_sphere, len(good_distances))

    dat_O2_p1_num_dens_values = f['O2_p1_number_density'][:][good_points] #Pulling out values at new indices within interp sphere
    dat_O2_p1_v_x = f['O2_p1_velocity_x'][:][good_points]
    dat_O2_p1_v_y = f['O2_p1_velocity_y'][:][good_points]
    dat_O2_p1_v_z = f['O2_p1_velocity_z'][:][good_points]
    dat_O2_p1_v_tot = np.sqrt(dat_O2_p1_v_x**2 + dat_O2_p1_v_y**2 + dat_O2_p1_v_z**2)

    dat_O_p1_num_dens_values = f['O_p1_number_density'][:][good_points]
    dat_O_p1_v_x = f['O_p1_velocity_x'][:][good_points]
    dat_O_p1_v_y = f['O_p1_velocity_y'][:][good_points]
    dat_O_p1_v_z = f['O_p1_velocity_z'][:][good_points]
    dat_O_p1_v_tot = np.sqrt(dat_O_p1_v_x**2 + dat_O_p1_v_y**2 + dat_O_p1_v_z**2)
    
    mag_field_x = f['magnetic_field_x'][:][good_points]
    mag_field_y = f['magnetic_field_y'][:][good_points]
    mag_field_z = f['magnetic_field_z'][:][good_points]
    mag_field_tot = np.sqrt(mag_field_x**2 + mag_field_y**2 + mag_field_z**2)
    
    p = np.array([good_x, good_y, good_z])
    normal = p/np.sqrt(np.sum(p**2, axis=0))
    norm = np.array([mag_field_x,mag_field_y,mag_field_z])
    mag_field_norm = np.sum(normal*norm, axis=0)
    
    interpolation_fields = np.array([dat_O2_p1_num_dens_values, dat_O2_p1_v_x, dat_O2_p1_v_y, dat_O2_p1_v_z, dat_O2_p1_v_tot,
                             dat_O_p1_num_dens_values, dat_O_p1_v_x, dat_O_p1_v_y, dat_O_p1_v_z, dat_O_p1_v_tot, 
                             mag_field_x, mag_field_y, mag_field_z, mag_field_tot, mag_field_norm])

    distances = good_distances #array of distances from center of sphere to points
    weights = 1.0 / (distances) #1/di factor

    for n in range(len(interpolation_fields)):
        numerator = np.sum(weights*interpolation_fields[n])
        denominator = np.sum(weights)
        interpolated_values_array = np.append(interpolated_values_array, (numerator / denominator))

reshaped_interpolated_values_array = np.reshape(interpolated_values_array, (len(interpolation_spheres_coords_cartesian[0]),15))

In [9]:
print np.min(number_of_points_in_sphere)
print np.max(number_of_points_in_sphere)

3.0
274.0


In [10]:
interpolation_sphere_coords_spherical = conversion_cart_to_sphere(interpolation_spheres_coords_cartesian[0], interpolation_spheres_coords_cartesian[1], interpolation_spheres_coords_cartesian[2])
stack_coords = np.column_stack(interpolation_sphere_coords_spherical)
stacked_interp_vals = np.hstack((reshaped_interpolated_values_array, stack_coords))

#SAVING values to txt file
file = open('LR_300km_interp_vals.txt','ab') #opens file to append without overwriting
#dataout = (reshaped_interpolated_values_array, interpolation_sphere_coords_spherical))
np.savetxt(file, stacked_interp_vals, delimiter=',',header=' O2+_num_dens, O2+_vx, O2+_vy, O2+_vz, O2+_vtot, O+_num_dens, O+_vx, O+_vy, O+_vz, O+_vtot, mag_x, mag_y, mag_z, mag_tot, mag_norm, r, theta, phi',
           fmt='%8.10e',comments='#') #saves columnated values as FWHM,xmin,xmax
file.close()

## V cross B values:

In [ ]:
r_sphere_300km = (3390.0+300.0) / 3390.0 #300km above surface
one_hundred_forty_five_km = (145.0) / 3390.0
r = r_sphere_300km
lon = np.arange(-90,271,1)
lat = np.arange(-90,91,1)

phi = -1*(lat-90)*np.pi/180.0
theta = lon*np.pi/180.0

phi_v, theta_v = np.meshgrid(phi, theta)

#Make face centers
phi_f = 0.5*(phi_v[1:,1:]+phi_v[:-1,:-1])
theta_f = 0.5*(theta_v[1:,1:]+theta_v[:-1,:-1])
lat_f = -1*phi_f*180/np.pi+90
lon_f = theta_f*180/np.pi

x = (r*np.cos(theta_f)*np.sin(phi_f)).flatten()
y = (r*np.sin(theta_f)*np.sin(phi_f)).flatten()    
z = (r*np.cos(phi_f)).flatten()
interpolation_spheres_coords_cartesian = np.array([x,y,z])

number_of_points_in_sphere = np.array([])
interpolated_values_array = np.array([])

for i in range(len(interpolation_spheres_coords_cartesian[0])):
    interp_r = np.sqrt(np.sum(interpolation_spheres_coords_cartesian[:,i]**2))
    r_dist = np.abs(dat_r_values - interp_r)
    distance = ((dat_x_values - interpolation_spheres_coords_cartesian[0,i])**2 +
                (dat_y_values - interpolation_spheres_coords_cartesian[1,i])**2 +
                (dat_z_values - interpolation_spheres_coords_cartesian[2,i])**2)
    
    good_points = np.logical_and(distance<=(one_hundred_forty_five_km**2),
                                r_dist<=15.0/3390.0)
    good_x, good_y, good_z, good_distances = dat_x_values[good_points], dat_y_values[good_points], dat_z_values[good_points], distance[good_points]
    number_of_points_in_sphere = np.append(number_of_points_in_sphere, len(good_distances))

    dat_H_p1_num_dens_values = f['H_p1_number_density'][:][good_points] #Pulling out values at new indices within interp sphere
    dat_H_p1_v_x = f['H_p1_velocity_x'][:][good_points]
    dat_H_p1_v_y = f['H_p1_velocity_y'][:][good_points]
    dat_H_p1_v_z = f['H_p1_velocity_z'][:][good_points]
    dat_H_p1_v_tot = np.sqrt(dat_H_p1_v_x**2 + dat_H_p1_v_y**2 + dat_H_p1_v_z**2)

    dat_velocity_x = f['velocity_x'][:][good_points] #background fluid velocities
    dat_velocity_y = f['velocity_y'][:][good_points]
    dat_velocity_z = f['velocity_z'][:][good_points]
    dat_velocity_total = np.sqrt(dat_velocity_x**2 + dat_velocity_y**2 + dat_velocity_z**2)
    
    dat_current_x = f['current_x'][:][good_points]
    dat_current_y = f['current_y'][:][good_points]
    dat_current_z = f['current_z'][:][good_points]
    dat_current_total = np.sqrt(dat_current_x**2 + dat_current_y**2 + dat_current_z**2)
    
    #J = np.array([f['current_x'][:][good_points], f['current_y'][:][good_points], f['current_z'][:][good_points]])
    #B = np.array([mag_field_x, mag_field_y, mag_field_z])
    #J_cross_B_x = J[1]*B[2] - J[2]*B[1]
    #J_cross_B_y = J[2]*B[0] - J[0]*B[2]
    #J_cross_B_z = J[0]*B[1] - J[1]*B[0]
    #J_cross_B_tot = np.sqrt(J_cross_B_x**2 + J_cross_B_y**2 + J_cross_B_z**2)
    
    interpolation_fields = np.array([dat_H_p1_num_dens_values, dat_H_p1_v_x, dat_H_p1_v_y, dat_H_p1_v_z, dat_H_p1_v_tot,
                                    dat_velocity_x, dat_velocity_y, dat_velocity_z, dat_velocity_total, dat_current_x,
                                    dat_current_y, dat_current_z, dat_current_total])#J_cross_B_x,J_cross_B_y,J_cross_B_z,J_cross_B_tot

    distances = good_distances #array of distances from center of sphere to points
    weights = 1.0 / (distances) #1/di factor

    for n in range(len(interpolation_fields)):
        numerator = np.sum(weights*interpolation_fields[n])
        denominator = np.sum(weights)
        interpolated_values_array = np.append(interpolated_values_array, (numerator / denominator))

reshaped_interpolated_values_array = np.reshape(interpolated_values_array, (len(interpolation_spheres_coords_cartesian[0]),13))

In [ ]:
#SAVING values to txt file
file = open('LR_300km_interp_vals_v_cross_b.txt','ab') #opens file to append without overwriting
#dataout = (reshaped_interpolated_values_array, interpolation_sphere_coords_spherical))
np.savetxt(file, reshaped_interpolated_values_array, delimiter=',',header=' H+_num_dens, H+_vx, H+_vy, H+_vz, H+_vtot, velocity_x, velocity_y, velocity_z, velocity_total, current_x, current_y, current_z, current_total',
           fmt='%8.10e',comments='#') #saves columnated values as FWHM,xmin,xmax
file.close()

# 400km:

In [11]:
r_sphere_400km = (3390.0+400.0) / 3390.0 #300km above surface
one_hundred_forty_five_km = (145.0) / 3390.0
r = r_sphere_400km
lon = np.arange(-90,271,1)
lat = np.arange(-90,91,1)

phi = -1*(lat-90)*np.pi/180.0
theta = lon*np.pi/180.0

phi_v, theta_v = np.meshgrid(phi, theta)

#Make face centers
phi_f = 0.5*(phi_v[1:,1:]+phi_v[:-1,:-1])
theta_f = 0.5*(theta_v[1:,1:]+theta_v[:-1,:-1])
lat_f = -1*phi_f*180/np.pi+90
lon_f = theta_f*180/np.pi

x = (r*np.cos(theta_f)*np.sin(phi_f)).flatten()
y = (r*np.sin(theta_f)*np.sin(phi_f)).flatten()    
z = (r*np.cos(phi_f)).flatten()
interpolation_spheres_coords_cartesian = np.array([x,y,z])

In [12]:
number_of_points_in_sphere = np.array([])
interpolated_values_array = np.array([])

for i in range(len(interpolation_spheres_coords_cartesian[0])):
    interp_r = np.sqrt(np.sum(interpolation_spheres_coords_cartesian[:,i]**2))
    r_dist = np.abs(dat_r_values - interp_r)
    distance = ((dat_x_values - interpolation_spheres_coords_cartesian[0,i])**2 +
                (dat_y_values - interpolation_spheres_coords_cartesian[1,i])**2 +
                (dat_z_values - interpolation_spheres_coords_cartesian[2,i])**2)
    
    good_points = np.logical_and(distance<=(one_hundred_forty_five_km**2),
                                r_dist<=25.0/3390.0)
    good_x, good_y, good_z, good_distances = dat_x_values[good_points], dat_y_values[good_points], dat_z_values[good_points], distance[good_points]
    number_of_points_in_sphere = np.append(number_of_points_in_sphere, len(good_distances))

    dat_O2_p1_num_dens_values = f['O2_p1_number_density'][:][good_points] #Pulling out values at new indices within interp sphere
    dat_O2_p1_v_x = f['O2_p1_velocity_x'][:][good_points]
    dat_O2_p1_v_y = f['O2_p1_velocity_y'][:][good_points]
    dat_O2_p1_v_z = f['O2_p1_velocity_z'][:][good_points]
    dat_O2_p1_v_tot = np.sqrt(dat_O2_p1_v_x**2 + dat_O2_p1_v_y**2 + dat_O2_p1_v_z**2)

    dat_O_p1_num_dens_values = f['O_p1_number_density'][:][good_points]
    dat_O_p1_v_x = f['O_p1_velocity_x'][:][good_points]
    dat_O_p1_v_y = f['O_p1_velocity_y'][:][good_points]
    dat_O_p1_v_z = f['O_p1_velocity_z'][:][good_points]
    dat_O_p1_v_tot = np.sqrt(dat_O_p1_v_x**2 + dat_O_p1_v_y**2 + dat_O_p1_v_z**2)
    
    mag_field_x = f['magnetic_field_x'][:][good_points]
    mag_field_y = f['magnetic_field_y'][:][good_points]
    mag_field_z = f['magnetic_field_z'][:][good_points]
    mag_field_tot = np.sqrt(mag_field_x**2 + mag_field_y**2 + mag_field_z**2)
    
    p = np.array([good_x, good_y, good_z])
    normal = p/np.sqrt(np.sum(p**2, axis=0))
    norm = np.array([mag_field_x,mag_field_y,mag_field_z])
    mag_field_norm = np.sum(normal*norm, axis=0)
    
    interpolation_fields = np.array([dat_O2_p1_num_dens_values, dat_O2_p1_v_x, dat_O2_p1_v_y, dat_O2_p1_v_z, dat_O2_p1_v_tot,
                             dat_O_p1_num_dens_values, dat_O_p1_v_x, dat_O_p1_v_y, dat_O_p1_v_z, dat_O_p1_v_tot, 
                             mag_field_x, mag_field_y, mag_field_z, mag_field_tot, mag_field_norm])

    distances = good_distances #array of distances from center of sphere to points
    weights = 1.0 / (distances) #1/di factor

    for n in range(len(interpolation_fields)):
        numerator = np.sum(weights*interpolation_fields[n])
        denominator = np.sum(weights)
        interpolated_values_array = np.append(interpolated_values_array, (numerator / denominator))

reshaped_interpolated_values_array = np.reshape(interpolated_values_array, (len(interpolation_spheres_coords_cartesian[0]),15))

In [13]:
print np.min(number_of_points_in_sphere)
print np.max(number_of_points_in_sphere)

5.0
448.0


In [14]:
interpolation_sphere_coords_spherical = conversion_cart_to_sphere(interpolation_spheres_coords_cartesian[0], interpolation_spheres_coords_cartesian[1], interpolation_spheres_coords_cartesian[2])
stack_coords = np.column_stack(interpolation_sphere_coords_spherical)
stacked_interp_vals = np.hstack((reshaped_interpolated_values_array, stack_coords))

#SAVING values to txt file
file = open('LR_400km_interp_vals.txt','ab') #opens file to append without overwriting
#dataout = (reshaped_interpolated_values_array, interpolation_sphere_coords_spherical))
np.savetxt(file, stacked_interp_vals, delimiter=',',header=' O2+_num_dens, O2+_vx, O2+_vy, O2+_vz, O2+_vtot, O+_num_dens, O+_vx, O+_vy, O+_vz, O+_vtot, mag_x, mag_y, mag_z, mag_tot, mag_norm, r, theta, phi',
           fmt='%8.10e',comments='#') #saves columnated values as FWHM,xmin,xmax
file.close()

## V cross B values:

In [ ]:
r_sphere_400km = (3390.0+400.0) / 3390.0 #300km above surface
one_hundred_forty_five_km = (145.0) / 3390.0
r = r_sphere_400km
lon = np.arange(-90,271,1)
lat = np.arange(-90,91,1)

phi = -1*(lat-90)*np.pi/180.0
theta = lon*np.pi/180.0

phi_v, theta_v = np.meshgrid(phi, theta)

#Make face centers
phi_f = 0.5*(phi_v[1:,1:]+phi_v[:-1,:-1])
theta_f = 0.5*(theta_v[1:,1:]+theta_v[:-1,:-1])
lat_f = -1*phi_f*180/np.pi+90
lon_f = theta_f*180/np.pi

x = (r*np.cos(theta_f)*np.sin(phi_f)).flatten()
y = (r*np.sin(theta_f)*np.sin(phi_f)).flatten()    
z = (r*np.cos(phi_f)).flatten()
interpolation_spheres_coords_cartesian = np.array([x,y,z])

number_of_points_in_sphere = np.array([])
interpolated_values_array = np.array([])

for i in range(len(interpolation_spheres_coords_cartesian[0])):
    interp_r = np.sqrt(np.sum(interpolation_spheres_coords_cartesian[:,i]**2))
    r_dist = np.abs(dat_r_values - interp_r)
    distance = ((dat_x_values - interpolation_spheres_coords_cartesian[0,i])**2 +
                (dat_y_values - interpolation_spheres_coords_cartesian[1,i])**2 +
                (dat_z_values - interpolation_spheres_coords_cartesian[2,i])**2)
    
    good_points = np.logical_and(distance<=(one_hundred_forty_five_km**2),
                                r_dist<=25.0/3390.0)
    good_x, good_y, good_z, good_distances = dat_x_values[good_points], dat_y_values[good_points], dat_z_values[good_points], distance[good_points]
    number_of_points_in_sphere = np.append(number_of_points_in_sphere, len(good_distances))

    dat_H_p1_num_dens_values = f['H_p1_number_density'][:][good_points] #Pulling out values at new indices within interp sphere
    dat_H_p1_v_x = f['H_p1_velocity_x'][:][good_points]
    dat_H_p1_v_y = f['H_p1_velocity_y'][:][good_points]
    dat_H_p1_v_z = f['H_p1_velocity_z'][:][good_points]
    dat_H_p1_v_tot = np.sqrt(dat_H_p1_v_x**2 + dat_H_p1_v_y**2 + dat_H_p1_v_z**2)

    dat_velocity_x = f['velocity_x'][:][good_points] #background fluid velocities
    dat_velocity_y = f['velocity_y'][:][good_points]
    dat_velocity_z = f['velocity_z'][:][good_points]
    dat_velocity_total = np.sqrt(dat_velocity_x**2 + dat_velocity_y**2 + dat_velocity_z**2)
    
    dat_current_x = f['current_x'][:][good_points]
    dat_current_y = f['current_y'][:][good_points]
    dat_current_z = f['current_z'][:][good_points]
    dat_current_total = np.sqrt(dat_current_x**2 + dat_current_y**2 + dat_current_z**2)
    
    #J = np.array([f['current_x'][:][good_points], f['current_y'][:][good_points], f['current_z'][:][good_points]])
    #B = np.array([mag_field_x, mag_field_y, mag_field_z])
    #J_cross_B_x = J[1]*B[2] - J[2]*B[1]
    #J_cross_B_y = J[2]*B[0] - J[0]*B[2]
    #J_cross_B_z = J[0]*B[1] - J[1]*B[0]
    #J_cross_B_tot = np.sqrt(J_cross_B_x**2 + J_cross_B_y**2 + J_cross_B_z**2)
    
    interpolation_fields = np.array([dat_H_p1_num_dens_values, dat_H_p1_v_x, dat_H_p1_v_y, dat_H_p1_v_z, dat_H_p1_v_tot,
                                    dat_velocity_x, dat_velocity_y, dat_velocity_z, dat_velocity_total, dat_current_x,
                                    dat_current_y, dat_current_z, dat_current_total])#J_cross_B_x,J_cross_B_y,J_cross_B_z,J_cross_B_tot

    distances = good_distances #array of distances from center of sphere to points
    weights = 1.0 / (distances) #1/di factor

    for n in range(len(interpolation_fields)):
        numerator = np.sum(weights*interpolation_fields[n])
        denominator = np.sum(weights)
        interpolated_values_array = np.append(interpolated_values_array, (numerator / denominator))

reshaped_interpolated_values_array = np.reshape(interpolated_values_array, (len(interpolation_spheres_coords_cartesian[0]),13))

In [ ]:
#SAVING values to txt file
file = open('LR_400km_interp_vals_v_cross_b.txt','ab') #opens file to append without overwriting
#dataout = (reshaped_interpolated_values_array, interpolation_sphere_coords_spherical))
np.savetxt(file, reshaped_interpolated_values_array, delimiter=',',header=' H+_num_dens, H+_vx, H+_vy, H+_vz, H+_vtot, velocity_x, velocity_y, velocity_z, velocity_total, current_x, current_y, current_z, current_total',
           fmt='%8.10e',comments='#') #saves columnated values as FWHM,xmin,xmax
file.close()

# 500km:

In [15]:
r_sphere_500km = (3390.0+500.0) / 3390.0 #300km above surface
one_hundred_forty_five_km = (145.0) / 3390.0
r = r_sphere_500km
lon = np.arange(-90,271,1)
lat = np.arange(-90,91,1)

phi = -1*(lat-90)*np.pi/180.0
theta = lon*np.pi/180.0

phi_v, theta_v = np.meshgrid(phi, theta)

#Make face centers
phi_f = 0.5*(phi_v[1:,1:]+phi_v[:-1,:-1])
theta_f = 0.5*(theta_v[1:,1:]+theta_v[:-1,:-1])
lat_f = -1*phi_f*180/np.pi+90
lon_f = theta_f*180/np.pi

x = (r*np.cos(theta_f)*np.sin(phi_f)).flatten()
y = (r*np.sin(theta_f)*np.sin(phi_f)).flatten()    
z = (r*np.cos(phi_f)).flatten()
interpolation_spheres_coords_cartesian = np.array([x,y,z])

In [16]:
number_of_points_in_sphere = np.array([])
interpolated_values_array = np.array([])

for i in range(len(interpolation_spheres_coords_cartesian[0])):
    interp_r = np.sqrt(np.sum(interpolation_spheres_coords_cartesian[:,i]**2))
    r_dist = np.abs(dat_r_values - interp_r)
    distance = ((dat_x_values - interpolation_spheres_coords_cartesian[0,i])**2 +
                (dat_y_values - interpolation_spheres_coords_cartesian[1,i])**2 +
                (dat_z_values - interpolation_spheres_coords_cartesian[2,i])**2)
    
    good_points = np.logical_and(distance<=(one_hundred_forty_five_km**2),
                                r_dist<=25.0/3390.0)
    good_x, good_y, good_z, good_distances = dat_x_values[good_points], dat_y_values[good_points], dat_z_values[good_points], distance[good_points]
    number_of_points_in_sphere = np.append(number_of_points_in_sphere, len(good_distances))

    dat_O2_p1_num_dens_values = f['O2_p1_number_density'][:][good_points] #Pulling out values at new indices within interp sphere
    dat_O2_p1_v_x = f['O2_p1_velocity_x'][:][good_points]
    dat_O2_p1_v_y = f['O2_p1_velocity_y'][:][good_points]
    dat_O2_p1_v_z = f['O2_p1_velocity_z'][:][good_points]
    dat_O2_p1_v_tot = np.sqrt(dat_O2_p1_v_x**2 + dat_O2_p1_v_y**2 + dat_O2_p1_v_z**2)

    dat_O_p1_num_dens_values = f['O_p1_number_density'][:][good_points]
    dat_O_p1_v_x = f['O_p1_velocity_x'][:][good_points]
    dat_O_p1_v_y = f['O_p1_velocity_y'][:][good_points]
    dat_O_p1_v_z = f['O_p1_velocity_z'][:][good_points]
    dat_O_p1_v_tot = np.sqrt(dat_O_p1_v_x**2 + dat_O_p1_v_y**2 + dat_O_p1_v_z**2)
    
    mag_field_x = f['magnetic_field_x'][:][good_points]
    mag_field_y = f['magnetic_field_y'][:][good_points]
    mag_field_z = f['magnetic_field_z'][:][good_points]
    mag_field_tot = np.sqrt(mag_field_x**2 + mag_field_y**2 + mag_field_z**2)
    
    p = np.array([good_x, good_y, good_z])
    normal = p/np.sqrt(np.sum(p**2, axis=0))
    norm = np.array([mag_field_x,mag_field_y,mag_field_z])
    mag_field_norm = np.sum(normal*norm, axis=0)
    
    interpolation_fields = np.array([dat_O2_p1_num_dens_values, dat_O2_p1_v_x, dat_O2_p1_v_y, dat_O2_p1_v_z, dat_O2_p1_v_tot,
                             dat_O_p1_num_dens_values, dat_O_p1_v_x, dat_O_p1_v_y, dat_O_p1_v_z, dat_O_p1_v_tot, 
                             mag_field_x, mag_field_y, mag_field_z, mag_field_tot, mag_field_norm])

    distances = good_distances #array of distances from center of sphere to points
    weights = 1.0 / (distances) #1/di factor

    for n in range(len(interpolation_fields)):
        numerator = np.sum(weights*interpolation_fields[n])
        denominator = np.sum(weights)
        interpolated_values_array = np.append(interpolated_values_array, (numerator / denominator))

reshaped_interpolated_values_array = np.reshape(interpolated_values_array, (len(interpolation_spheres_coords_cartesian[0]),15))

In [17]:
print np.min(number_of_points_in_sphere)
print np.max(number_of_points_in_sphere)

5.0
440.0


In [18]:
interpolation_sphere_coords_spherical = conversion_cart_to_sphere(interpolation_spheres_coords_cartesian[0], interpolation_spheres_coords_cartesian[1], interpolation_spheres_coords_cartesian[2])
stack_coords = np.column_stack(interpolation_sphere_coords_spherical)
stacked_interp_vals = np.hstack((reshaped_interpolated_values_array, stack_coords))

#SAVING values to txt file
file = open('LR_500km_interp_vals.txt','ab') #opens file to append without overwriting
#dataout = (reshaped_interpolated_values_array, interpolation_sphere_coords_spherical))
np.savetxt(file, stacked_interp_vals, delimiter=',',header=' O2+_num_dens, O2+_vx, O2+_vy, O2+_vz, O2+_vtot, O+_num_dens, O+_vx, O+_vy, O+_vz, O+_vtot, mag_x, mag_y, mag_z, mag_tot, mag_norm, r, theta, phi',
           fmt='%8.10e',comments='#') #saves columnated values as FWHM,xmin,xmax
file.close()

## V cross B values: 

In [ ]:
r_sphere_500km = (3390.0+500.0) / 3390.0 #300km above surface
one_hundred_forty_five_km = (145.0) / 3390.0
r = r_sphere_500km
lon = np.arange(-90,271,1)
lat = np.arange(-90,91,1)

phi = -1*(lat-90)*np.pi/180.0
theta = lon*np.pi/180.0

phi_v, theta_v = np.meshgrid(phi, theta)

#Make face centers
phi_f = 0.5*(phi_v[1:,1:]+phi_v[:-1,:-1])
theta_f = 0.5*(theta_v[1:,1:]+theta_v[:-1,:-1])
lat_f = -1*phi_f*180/np.pi+90
lon_f = theta_f*180/np.pi

x = (r*np.cos(theta_f)*np.sin(phi_f)).flatten()
y = (r*np.sin(theta_f)*np.sin(phi_f)).flatten()    
z = (r*np.cos(phi_f)).flatten()
interpolation_spheres_coords_cartesian = np.array([x,y,z])

number_of_points_in_sphere = np.array([])
interpolated_values_array = np.array([])

for i in range(len(interpolation_spheres_coords_cartesian[0])):
    interp_r = np.sqrt(np.sum(interpolation_spheres_coords_cartesian[:,i]**2))
    r_dist = np.abs(dat_r_values - interp_r)
    distance = ((dat_x_values - interpolation_spheres_coords_cartesian[0,i])**2 +
                (dat_y_values - interpolation_spheres_coords_cartesian[1,i])**2 +
                (dat_z_values - interpolation_spheres_coords_cartesian[2,i])**2)
    
    good_points = np.logical_and(distance<=(one_hundred_forty_five_km**2),
                                r_dist<=25.0/3390.0)
    good_x, good_y, good_z, good_distances = dat_x_values[good_points], dat_y_values[good_points], dat_z_values[good_points], distance[good_points]
    number_of_points_in_sphere = np.append(number_of_points_in_sphere, len(good_distances))

    dat_H_p1_num_dens_values = f['H_p1_number_density'][:][good_points] #Pulling out values at new indices within interp sphere
    dat_H_p1_v_x = f['H_p1_velocity_x'][:][good_points]
    dat_H_p1_v_y = f['H_p1_velocity_y'][:][good_points]
    dat_H_p1_v_z = f['H_p1_velocity_z'][:][good_points]
    dat_H_p1_v_tot = np.sqrt(dat_H_p1_v_x**2 + dat_H_p1_v_y**2 + dat_H_p1_v_z**2)

    dat_velocity_x = f['velocity_x'][:][good_points] #background fluid velocities
    dat_velocity_y = f['velocity_y'][:][good_points]
    dat_velocity_z = f['velocity_z'][:][good_points]
    dat_velocity_total = np.sqrt(dat_velocity_x**2 + dat_velocity_y**2 + dat_velocity_z**2)
    
    dat_current_x = f['current_x'][:][good_points]
    dat_current_y = f['current_y'][:][good_points]
    dat_current_z = f['current_z'][:][good_points]
    dat_current_total = np.sqrt(dat_current_x**2 + dat_current_y**2 + dat_current_z**2)
    
    #J = np.array([f['current_x'][:][good_points], f['current_y'][:][good_points], f['current_z'][:][good_points]])
    #B = np.array([mag_field_x, mag_field_y, mag_field_z])
    #J_cross_B_x = J[1]*B[2] - J[2]*B[1]
    #J_cross_B_y = J[2]*B[0] - J[0]*B[2]
    #J_cross_B_z = J[0]*B[1] - J[1]*B[0]
    #J_cross_B_tot = np.sqrt(J_cross_B_x**2 + J_cross_B_y**2 + J_cross_B_z**2)
    
    interpolation_fields = np.array([dat_H_p1_num_dens_values, dat_H_p1_v_x, dat_H_p1_v_y, dat_H_p1_v_z, dat_H_p1_v_tot,
                                    dat_velocity_x, dat_velocity_y, dat_velocity_z, dat_velocity_total, dat_current_x,
                                    dat_current_y, dat_current_z, dat_current_total])#J_cross_B_x,J_cross_B_y,J_cross_B_z,J_cross_B_tot

    distances = good_distances #array of distances from center of sphere to points
    weights = 1.0 / (distances) #1/di factor

    for n in range(len(interpolation_fields)):
        numerator = np.sum(weights*interpolation_fields[n])
        denominator = np.sum(weights)
        interpolated_values_array = np.append(interpolated_values_array, (numerator / denominator))

reshaped_interpolated_values_array = np.reshape(interpolated_values_array, (len(interpolation_spheres_coords_cartesian[0]),13))

In [ ]:
#SAVING values to txt file
file = open('LR_500km_interp_vals_v_cross_b.txt','ab') #opens file to append without overwriting
#dataout = (reshaped_interpolated_values_array, interpolation_sphere_coords_spherical))
np.savetxt(file, reshaped_interpolated_values_array, delimiter=',',header=' H+_num_dens, H+_vx, H+_vy, H+_vz, H+_vtot, velocity_x, velocity_y, velocity_z, velocity_total, current_x, current_y, current_z, current_total',
           fmt='%8.10e',comments='#') #saves columnated values as FWHM,xmin,xmax
file.close()

# 600km:

In [19]:
r_sphere_600km = (3390.0+600.0) / 3390.0 #300km above surface
one_hundred_fifty_km = (150.0) / 3390.0
r = r_sphere_600km
lon = np.arange(-90,271,1)
lat = np.arange(-90,91,1)

phi = -1*(lat-90)*np.pi/180.0
theta = lon*np.pi/180.0

phi_v, theta_v = np.meshgrid(phi, theta)

#Make face centers
phi_f = 0.5*(phi_v[1:,1:]+phi_v[:-1,:-1])
theta_f = 0.5*(theta_v[1:,1:]+theta_v[:-1,:-1])
lat_f = -1*phi_f*180/np.pi+90
lon_f = theta_f*180/np.pi

x = (r*np.cos(theta_f)*np.sin(phi_f)).flatten()
y = (r*np.sin(theta_f)*np.sin(phi_f)).flatten()
z = (r*np.cos(phi_f)).flatten()
interpolation_spheres_coords_cartesian = np.array([x,y,z])

In [20]:
number_of_points_in_sphere = np.array([])
interpolated_values_array = np.array([])

for i in range(len(interpolation_spheres_coords_cartesian[0])):
    interp_r = np.sqrt(np.sum(interpolation_spheres_coords_cartesian[:,i]**2))
    r_dist = np.abs(dat_r_values - interp_r)
    distance = ((dat_x_values - interpolation_spheres_coords_cartesian[0,i])**2 +
                (dat_y_values - interpolation_spheres_coords_cartesian[1,i])**2 +
                (dat_z_values - interpolation_spheres_coords_cartesian[2,i])**2)
    
    good_points = np.logical_and(distance<=(one_hundred_fifty_km**2),
                                r_dist<=25.0/3390.0)
    good_x, good_y, good_z, good_distances = dat_x_values[good_points], dat_y_values[good_points], dat_z_values[good_points], distance[good_points]
    number_of_points_in_sphere = np.append(number_of_points_in_sphere, len(good_distances))

    dat_O2_p1_num_dens_values = f['O2_p1_number_density'][:][good_points] #Pulling out values at new indices within interp sphere
    dat_O2_p1_v_x = f['O2_p1_velocity_x'][:][good_points]
    dat_O2_p1_v_y = f['O2_p1_velocity_y'][:][good_points]
    dat_O2_p1_v_z = f['O2_p1_velocity_z'][:][good_points]
    dat_O2_p1_v_tot = np.sqrt(dat_O2_p1_v_x**2 + dat_O2_p1_v_y**2 + dat_O2_p1_v_z**2)

    dat_O_p1_num_dens_values = f['O_p1_number_density'][:][good_points]
    dat_O_p1_v_x = f['O_p1_velocity_x'][:][good_points]
    dat_O_p1_v_y = f['O_p1_velocity_y'][:][good_points]
    dat_O_p1_v_z = f['O_p1_velocity_z'][:][good_points]
    dat_O_p1_v_tot = np.sqrt(dat_O_p1_v_x**2 + dat_O_p1_v_y**2 + dat_O_p1_v_z**2)
    
    mag_field_x = f['magnetic_field_x'][:][good_points]
    mag_field_y = f['magnetic_field_y'][:][good_points]
    mag_field_z = f['magnetic_field_z'][:][good_points]
    mag_field_tot = np.sqrt(mag_field_x**2 + mag_field_y**2 + mag_field_z**2)
    
    p = np.array([good_x, good_y, good_z])
    normal = p/np.sqrt(np.sum(p**2, axis=0))
    norm = np.array([mag_field_x,mag_field_y,mag_field_z])
    mag_field_norm = np.sum(normal*norm, axis=0)
    
    interpolation_fields = np.array([dat_O2_p1_num_dens_values, dat_O2_p1_v_x, dat_O2_p1_v_y, dat_O2_p1_v_z, dat_O2_p1_v_tot,
                             dat_O_p1_num_dens_values, dat_O_p1_v_x, dat_O_p1_v_y, dat_O_p1_v_z, dat_O_p1_v_tot, 
                             mag_field_x, mag_field_y, mag_field_z, mag_field_tot, mag_field_norm])

    distances = good_distances #array of distances from center of sphere to points
    weights = 1.0 / (distances) #1/di factor

    for n in range(len(interpolation_fields)):
        numerator = np.sum(weights*interpolation_fields[n])
        denominator = np.sum(weights)
        interpolated_values_array = np.append(interpolated_values_array, (numerator / denominator))

reshaped_interpolated_values_array = np.reshape(interpolated_values_array, (len(interpolation_spheres_coords_cartesian[0]),15))

In [21]:
print np.min(number_of_points_in_sphere)
print np.max(number_of_points_in_sphere)

4.0
354.0


In [22]:
interpolation_sphere_coords_spherical = conversion_cart_to_sphere(interpolation_spheres_coords_cartesian[0], interpolation_spheres_coords_cartesian[1], interpolation_spheres_coords_cartesian[2])
stack_coords = np.column_stack(interpolation_sphere_coords_spherical)
stacked_interp_vals = np.hstack((reshaped_interpolated_values_array, stack_coords))

#SAVING values to txt file
file = open('LR_600km_interp_vals.txt','ab') #opens file to append without overwriting
#dataout = (reshaped_interpolated_values_array, interpolation_sphere_coords_spherical))
np.savetxt(file, stacked_interp_vals, delimiter=',',header=' O2+_num_dens, O2+_vx, O2+_vy, O2+_vz, O2+_vtot, O+_num_dens, O+_vx, O+_vy, O+_vz, O+_vtot, mag_x, mag_y, mag_z, mag_tot, mag_norm, r, theta, phi',
           fmt='%8.10e',comments='#') #saves columnated values as FWHM,xmin,xmax
file.close()

## V cross B values: 

In [ ]:
r_sphere_600km = (3390.0+600.0) / 3390.0 #300km above surface
one_hundred_fifty_km = (150.0) / 3390.0
r = r_sphere_600km
lon = np.arange(-90,271,1)
lat = np.arange(-90,91,1)

phi = -1*(lat-90)*np.pi/180.0
theta = lon*np.pi/180.0

phi_v, theta_v = np.meshgrid(phi, theta)

#Make face centers
phi_f = 0.5*(phi_v[1:,1:]+phi_v[:-1,:-1])
theta_f = 0.5*(theta_v[1:,1:]+theta_v[:-1,:-1])
lat_f = -1*phi_f*180/np.pi+90
lon_f = theta_f*180/np.pi

x = (r*np.cos(theta_f)*np.sin(phi_f)).flatten()
y = (r*np.sin(theta_f)*np.sin(phi_f)).flatten()
z = (r*np.cos(phi_f)).flatten()
interpolation_spheres_coords_cartesian = np.array([x,y,z])

number_of_points_in_sphere = np.array([])
interpolated_values_array = np.array([])

for i in range(len(interpolation_spheres_coords_cartesian[0])):
    interp_r = np.sqrt(np.sum(interpolation_spheres_coords_cartesian[:,i]**2))
    r_dist = np.abs(dat_r_values - interp_r)
    distance = ((dat_x_values - interpolation_spheres_coords_cartesian[0,i])**2 +
                (dat_y_values - interpolation_spheres_coords_cartesian[1,i])**2 +
                (dat_z_values - interpolation_spheres_coords_cartesian[2,i])**2)
    
    good_points = np.logical_and(distance<=(one_hundred_fifty_km**2),
                                r_dist<=25.0/3390.0)
    good_x, good_y, good_z, good_distances = dat_x_values[good_points], dat_y_values[good_points], dat_z_values[good_points], distance[good_points]
    number_of_points_in_sphere = np.append(number_of_points_in_sphere, len(good_distances))

    dat_H_p1_num_dens_values = f['H_p1_number_density'][:][good_points] #Pulling out values at new indices within interp sphere
    dat_H_p1_v_x = f['H_p1_velocity_x'][:][good_points]
    dat_H_p1_v_y = f['H_p1_velocity_y'][:][good_points]
    dat_H_p1_v_z = f['H_p1_velocity_z'][:][good_points]
    dat_H_p1_v_tot = np.sqrt(dat_H_p1_v_x**2 + dat_H_p1_v_y**2 + dat_H_p1_v_z**2)

    dat_velocity_x = f['velocity_x'][:][good_points] #background fluid velocities
    dat_velocity_y = f['velocity_y'][:][good_points]
    dat_velocity_z = f['velocity_z'][:][good_points]
    dat_velocity_total = np.sqrt(dat_velocity_x**2 + dat_velocity_y**2 + dat_velocity_z**2)
    
    dat_current_x = f['current_x'][:][good_points]
    dat_current_y = f['current_y'][:][good_points]
    dat_current_z = f['current_z'][:][good_points]
    dat_current_total = np.sqrt(dat_current_x**2 + dat_current_y**2 + dat_current_z**2)
    
    #J = np.array([f['current_x'][:][good_points], f['current_y'][:][good_points], f['current_z'][:][good_points]])
    #B = np.array([mag_field_x, mag_field_y, mag_field_z])
    #J_cross_B_x = J[1]*B[2] - J[2]*B[1]
    #J_cross_B_y = J[2]*B[0] - J[0]*B[2]
    #J_cross_B_z = J[0]*B[1] - J[1]*B[0]
    #J_cross_B_tot = np.sqrt(J_cross_B_x**2 + J_cross_B_y**2 + J_cross_B_z**2)
    
    interpolation_fields = np.array([dat_H_p1_num_dens_values, dat_H_p1_v_x, dat_H_p1_v_y, dat_H_p1_v_z, dat_H_p1_v_tot,
                                    dat_velocity_x, dat_velocity_y, dat_velocity_z, dat_velocity_total, dat_current_x,
                                    dat_current_y, dat_current_z, dat_current_total])#J_cross_B_x,J_cross_B_y,J_cross_B_z,J_cross_B_tot

    distances = good_distances #array of distances from center of sphere to points
    weights = 1.0 / (distances) #1/di factor

    for n in range(len(interpolation_fields)):
        numerator = np.sum(weights*interpolation_fields[n])
        denominator = np.sum(weights)
        interpolated_values_array = np.append(interpolated_values_array, (numerator / denominator))

reshaped_interpolated_values_array = np.reshape(interpolated_values_array, (len(interpolation_spheres_coords_cartesian[0]),13))

In [ ]:
#SAVING values to txt file
file = open('LR_600km_interp_vals_v_cross_b.txt','ab') #opens file to append without overwriting
#dataout = (reshaped_interpolated_values_array, interpolation_sphere_coords_spherical))
np.savetxt(file, reshaped_interpolated_values_array, delimiter=',',header=' H+_num_dens, H+_vx, H+_vy, H+_vz, H+_vtot, velocity_x, velocity_y, velocity_z, velocity_total, current_x, current_y, current_z, current_total',
           fmt='%8.10e',comments='#') #saves columnated values as FWHM,xmin,xmax
file.close()

# 700km:

In [23]:
r_sphere_700km = (3390.0+700.0) / 3390.0 #300km above surface
one_hundred_fifty_five_km = (155.0) / 3390.0
r = r_sphere_700km
lon = np.arange(-90,271,1)
lat = np.arange(-90,91,1)

phi = -1*(lat-90)*np.pi/180.0
theta = lon*np.pi/180.0

phi_v, theta_v = np.meshgrid(phi, theta)

#Make face centers
phi_f = 0.5*(phi_v[1:,1:]+phi_v[:-1,:-1])
theta_f = 0.5*(theta_v[1:,1:]+theta_v[:-1,:-1])
lat_f = -1*phi_f*180/np.pi+90
lon_f = theta_f*180/np.pi

x = (r*np.cos(theta_f)*np.sin(phi_f)).flatten()
y = (r*np.sin(theta_f)*np.sin(phi_f)).flatten()
z = (r*np.cos(phi_f)).flatten()
interpolation_spheres_coords_cartesian = np.array([x,y,z])

In [24]:
number_of_points_in_sphere = np.array([])
interpolated_values_array = np.array([])

for i in range(len(interpolation_spheres_coords_cartesian[0])):
    interp_r = np.sqrt(np.sum(interpolation_spheres_coords_cartesian[:,i]**2))
    r_dist = np.abs(dat_r_values - interp_r)
    distance = ((dat_x_values - interpolation_spheres_coords_cartesian[0,i])**2 +
                (dat_y_values - interpolation_spheres_coords_cartesian[1,i])**2 +
                (dat_z_values - interpolation_spheres_coords_cartesian[2,i])**2)
    
    good_points = np.logical_and(distance<=(one_hundred_fifty_five_km**2),
                                r_dist<=25.0/3390.0)
    good_x, good_y, good_z, good_distances = dat_x_values[good_points], dat_y_values[good_points], dat_z_values[good_points], distance[good_points]
    number_of_points_in_sphere = np.append(number_of_points_in_sphere, len(good_distances))

    dat_O2_p1_num_dens_values = f['O2_p1_number_density'][:][good_points] #Pulling out values at new indices within interp sphere
    dat_O2_p1_v_x = f['O2_p1_velocity_x'][:][good_points]
    dat_O2_p1_v_y = f['O2_p1_velocity_y'][:][good_points]
    dat_O2_p1_v_z = f['O2_p1_velocity_z'][:][good_points]
    dat_O2_p1_v_tot = np.sqrt(dat_O2_p1_v_x**2 + dat_O2_p1_v_y**2 + dat_O2_p1_v_z**2)

    dat_O_p1_num_dens_values = f['O_p1_number_density'][:][good_points]
    dat_O_p1_v_x = f['O_p1_velocity_x'][:][good_points]
    dat_O_p1_v_y = f['O_p1_velocity_y'][:][good_points]
    dat_O_p1_v_z = f['O_p1_velocity_z'][:][good_points]
    dat_O_p1_v_tot = np.sqrt(dat_O_p1_v_x**2 + dat_O_p1_v_y**2 + dat_O_p1_v_z**2)
    
    mag_field_x = f['magnetic_field_x'][:][good_points]
    mag_field_y = f['magnetic_field_y'][:][good_points]
    mag_field_z = f['magnetic_field_z'][:][good_points]
    mag_field_tot = np.sqrt(mag_field_x**2 + mag_field_y**2 + mag_field_z**2)
    
    p = np.array([good_x, good_y, good_z])
    normal = p/np.sqrt(np.sum(p**2, axis=0))
    norm = np.array([mag_field_x,mag_field_y,mag_field_z])
    mag_field_norm = np.sum(normal*norm, axis=0)
    
    interpolation_fields = np.array([dat_O2_p1_num_dens_values, dat_O2_p1_v_x, dat_O2_p1_v_y, dat_O2_p1_v_z, dat_O2_p1_v_tot,
                             dat_O_p1_num_dens_values, dat_O_p1_v_x, dat_O_p1_v_y, dat_O_p1_v_z, dat_O_p1_v_tot, 
                             mag_field_x, mag_field_y, mag_field_z, mag_field_tot, mag_field_norm])

    distances = good_distances #array of distances from center of sphere to points
    weights = 1.0 / (distances) #1/di factor

    for n in range(len(interpolation_fields)):
        numerator = np.sum(weights*interpolation_fields[n])
        denominator = np.sum(weights)
        interpolated_values_array = np.append(interpolated_values_array, (numerator / denominator))

reshaped_interpolated_values_array = np.reshape(interpolated_values_array, (len(interpolation_spheres_coords_cartesian[0]),15))

In [25]:
print np.min(number_of_points_in_sphere)
print np.max(number_of_points_in_sphere)

2.0
178.0


In [26]:
interpolation_sphere_coords_spherical = conversion_cart_to_sphere(interpolation_spheres_coords_cartesian[0], interpolation_spheres_coords_cartesian[1], interpolation_spheres_coords_cartesian[2])
stack_coords = np.column_stack(interpolation_sphere_coords_spherical)
stacked_interp_vals = np.hstack((reshaped_interpolated_values_array, stack_coords))

#SAVING values to txt file
file = open('LR_700km_interp_vals.txt','ab') #opens file to append without overwriting
#dataout = (reshaped_interpolated_values_array, interpolation_sphere_coords_spherical))
np.savetxt(file, stacked_interp_vals, delimiter=',',header=' O2+_num_dens, O2+_vx, O2+_vy, O2+_vz, O2+_vtot, O+_num_dens, O+_vx, O+_vy, O+_vz, O+_vtot, mag_x, mag_y, mag_z, mag_tot, mag_norm, r, theta, phi',
           fmt='%8.10e',comments='#') #saves columnated values as FWHM,xmin,xmax
file.close()

## V cross B values: 

In [ ]:
r_sphere_700km = (3390.0+700.0) / 3390.0 #300km above surface
one_hundred_fifty_five_km = (155.0) / 3390.0
r = r_sphere_700km
lon = np.arange(-90,271,1)
lat = np.arange(-90,91,1)

phi = -1*(lat-90)*np.pi/180.0
theta = lon*np.pi/180.0

phi_v, theta_v = np.meshgrid(phi, theta)

#Make face centers
phi_f = 0.5*(phi_v[1:,1:]+phi_v[:-1,:-1])
theta_f = 0.5*(theta_v[1:,1:]+theta_v[:-1,:-1])
lat_f = -1*phi_f*180/np.pi+90
lon_f = theta_f*180/np.pi

x = (r*np.cos(theta_f)*np.sin(phi_f)).flatten()
y = (r*np.sin(theta_f)*np.sin(phi_f)).flatten()
z = (r*np.cos(phi_f)).flatten()
interpolation_spheres_coords_cartesian = np.array([x,y,z])

number_of_points_in_sphere = np.array([])
interpolated_values_array = np.array([])

for i in range(len(interpolation_spheres_coords_cartesian[0])):
    interp_r = np.sqrt(np.sum(interpolation_spheres_coords_cartesian[:,i]**2))
    r_dist = np.abs(dat_r_values - interp_r)
    distance = ((dat_x_values - interpolation_spheres_coords_cartesian[0,i])**2 +
                (dat_y_values - interpolation_spheres_coords_cartesian[1,i])**2 +
                (dat_z_values - interpolation_spheres_coords_cartesian[2,i])**2)
    
    good_points = np.logical_and(distance<=(one_hundred_fifty_five_km**2),
                                r_dist<=25.0/3390.0)
    good_x, good_y, good_z, good_distances = dat_x_values[good_points], dat_y_values[good_points], dat_z_values[good_points], distance[good_points]
    number_of_points_in_sphere = np.append(number_of_points_in_sphere, len(good_distances))

    dat_H_p1_num_dens_values = f['H_p1_number_density'][:][good_points] #Pulling out values at new indices within interp sphere
    dat_H_p1_v_x = f['H_p1_velocity_x'][:][good_points]
    dat_H_p1_v_y = f['H_p1_velocity_y'][:][good_points]
    dat_H_p1_v_z = f['H_p1_velocity_z'][:][good_points]
    dat_H_p1_v_tot = np.sqrt(dat_H_p1_v_x**2 + dat_H_p1_v_y**2 + dat_H_p1_v_z**2)

    dat_velocity_x = f['velocity_x'][:][good_points] #background fluid velocities
    dat_velocity_y = f['velocity_y'][:][good_points]
    dat_velocity_z = f['velocity_z'][:][good_points]
    dat_velocity_total = np.sqrt(dat_velocity_x**2 + dat_velocity_y**2 + dat_velocity_z**2)
    
    dat_current_x = f['current_x'][:][good_points]
    dat_current_y = f['current_y'][:][good_points]
    dat_current_z = f['current_z'][:][good_points]
    dat_current_total = np.sqrt(dat_current_x**2 + dat_current_y**2 + dat_current_z**2)
    
    #J = np.array([f['current_x'][:][good_points], f['current_y'][:][good_points], f['current_z'][:][good_points]])
    #B = np.array([mag_field_x, mag_field_y, mag_field_z])
    #J_cross_B_x = J[1]*B[2] - J[2]*B[1]
    #J_cross_B_y = J[2]*B[0] - J[0]*B[2]
    #J_cross_B_z = J[0]*B[1] - J[1]*B[0]
    #J_cross_B_tot = np.sqrt(J_cross_B_x**2 + J_cross_B_y**2 + J_cross_B_z**2)
    
    interpolation_fields = np.array([dat_H_p1_num_dens_values, dat_H_p1_v_x, dat_H_p1_v_y, dat_H_p1_v_z, dat_H_p1_v_tot,
                                    dat_velocity_x, dat_velocity_y, dat_velocity_z, dat_velocity_total, dat_current_x,
                                    dat_current_y, dat_current_z, dat_current_total])#J_cross_B_x,J_cross_B_y,J_cross_B_z,J_cross_B_tot

    distances = good_distances #array of distances from center of sphere to points
    weights = 1.0 / (distances) #1/di factor

    for n in range(len(interpolation_fields)):
        numerator = np.sum(weights*interpolation_fields[n])
        denominator = np.sum(weights)
        interpolated_values_array = np.append(interpolated_values_array, (numerator / denominator))

reshaped_interpolated_values_array = np.reshape(interpolated_values_array, (len(interpolation_spheres_coords_cartesian[0]),13))

In [ ]:
#SAVING values to txt file
file = open('LR_700km_interp_vals_v_cross_b.txt','ab') #opens file to append without overwriting
#dataout = (reshaped_interpolated_values_array, interpolation_sphere_coords_spherical))
np.savetxt(file, reshaped_interpolated_values_array, delimiter=',',header=' H+_num_dens, H+_vx, H+_vy, H+_vz, H+_vtot, velocity_x, velocity_y, velocity_z, velocity_total, current_x, current_y, current_z, current_total',
           fmt='%8.10e',comments='#') #saves columnated values as FWHM,xmin,xmax
file.close()

# 800km:

In [27]:
r_sphere_800km = (3390.0+800.0) / 3390.0 #300km above surface
one_hundred_sixty_km = (160.0) / 3390.0
r = r_sphere_800km
lon = np.arange(-90,272,1)
lat = np.arange(-90,91,1)

phi = -1*(lat-90)*np.pi/180.0
theta = lon*np.pi/180.0

phi_v, theta_v = np.meshgrid(phi, theta)

#Make face centers
phi_f = 0.5*(phi_v[1:,1:]+phi_v[:-1,:-1])
theta_f = 0.5*(theta_v[1:,1:]+theta_v[:-1,:-1])
lat_f = -1*phi_f*180/np.pi+90
lon_f = theta_f*180/np.pi

x = (r*np.cos(theta_f)*np.sin(phi_f)).flatten()
y = (r*np.sin(theta_f)*np.sin(phi_f)).flatten()
z = (r*np.cos(phi_f)).flatten()
interpolation_spheres_coords_cartesian = np.array([x,y,z])

In [28]:
number_of_points_in_sphere = np.array([])
interpolated_values_array = np.array([])

for i in range(len(interpolation_spheres_coords_cartesian[0])):
    interp_r = np.sqrt(np.sum(interpolation_spheres_coords_cartesian[:,i]**2))
    r_dist = np.abs(dat_r_values - interp_r)
    distance = ((dat_x_values - interpolation_spheres_coords_cartesian[0,i])**2 +
                (dat_y_values - interpolation_spheres_coords_cartesian[1,i])**2 +
                (dat_z_values - interpolation_spheres_coords_cartesian[2,i])**2)
    
    good_points = np.logical_and(distance<=(one_hundred_sixty_km**2),
                                r_dist<=25.0/3390.0)
    good_x, good_y, good_z, good_distances = dat_x_values[good_points], dat_y_values[good_points], dat_z_values[good_points], distance[good_points]
    number_of_points_in_sphere = np.append(number_of_points_in_sphere, len(good_distances))

    dat_O2_p1_num_dens_values = f['O2_p1_number_density'][:][good_points] #Pulling out values at new indices within interp sphere
    dat_O2_p1_v_x = f['O2_p1_velocity_x'][:][good_points]
    dat_O2_p1_v_y = f['O2_p1_velocity_y'][:][good_points]
    dat_O2_p1_v_z = f['O2_p1_velocity_z'][:][good_points]
    dat_O2_p1_v_tot = np.sqrt(dat_O2_p1_v_x**2 + dat_O2_p1_v_y**2 + dat_O2_p1_v_z**2)

    dat_O_p1_num_dens_values = f['O_p1_number_density'][:][good_points]
    dat_O_p1_v_x = f['O_p1_velocity_x'][:][good_points]
    dat_O_p1_v_y = f['O_p1_velocity_y'][:][good_points]
    dat_O_p1_v_z = f['O_p1_velocity_z'][:][good_points]
    dat_O_p1_v_tot = np.sqrt(dat_O_p1_v_x**2 + dat_O_p1_v_y**2 + dat_O_p1_v_z**2)
    
    mag_field_x = f['magnetic_field_x'][:][good_points]
    mag_field_y = f['magnetic_field_y'][:][good_points]
    mag_field_z = f['magnetic_field_z'][:][good_points]
    mag_field_tot = np.sqrt(mag_field_x**2 + mag_field_y**2 + mag_field_z**2)
    
    p = np.array([good_x, good_y, good_z])
    normal = p/np.sqrt(np.sum(p**2, axis=0))
    norm = np.array([mag_field_x,mag_field_y,mag_field_z])
    mag_field_norm = np.sum(normal*norm, axis=0)
    
    interpolation_fields = np.array([dat_O2_p1_num_dens_values, dat_O2_p1_v_x, dat_O2_p1_v_y, dat_O2_p1_v_z, dat_O2_p1_v_tot,
                             dat_O_p1_num_dens_values, dat_O_p1_v_x, dat_O_p1_v_y, dat_O_p1_v_z, dat_O_p1_v_tot, 
                             mag_field_x, mag_field_y, mag_field_z, mag_field_tot, mag_field_norm])

    distances = good_distances #array of distances from center of sphere to points
    weights = 1.0 / (distances) #1/di factor

    for n in range(len(interpolation_fields)):
        numerator = np.sum(weights*interpolation_fields[n])
        denominator = np.sum(weights)
        interpolated_values_array = np.append(interpolated_values_array, (numerator / denominator))

reshaped_interpolated_values_array = np.reshape(interpolated_values_array, (len(interpolation_spheres_coords_cartesian[0]),15))

In [29]:
print np.min(number_of_points_in_sphere)
print np.max(number_of_points_in_sphere)

2.0
178.0


In [30]:
interpolation_sphere_coords_spherical = conversion_cart_to_sphere(interpolation_spheres_coords_cartesian[0], interpolation_spheres_coords_cartesian[1], interpolation_spheres_coords_cartesian[2])
stack_coords = np.column_stack(interpolation_sphere_coords_spherical)
stacked_interp_vals = np.hstack((reshaped_interpolated_values_array, stack_coords))

#SAVING values to txt file
file = open('LR_800km_interp_vals.txt','ab') #opens file to append without overwriting
#dataout = (reshaped_interpolated_values_array, interpolation_sphere_coords_spherical))
np.savetxt(file, stacked_interp_vals, delimiter=',',header=' O2+_num_dens, O2+_vx, O2+_vy, O2+_vz, O2+_vtot, O+_num_dens, O+_vx, O+_vy, O+_vz, O+_vtot, mag_x, mag_y, mag_z, mag_tot, mag_norm, r, theta, phi',
           fmt='%8.10e',comments='#') #saves columnated values as FWHM,xmin,xmax
file.close()

## V cross B values: 

In [ ]:
r_sphere_800km = (3390.0+800.0) / 3390.0 #300km above surface
one_hundred_sixty_km = (160.0) / 3390.0
r = r_sphere_800km
lon = np.arange(-90,272,1)
lat = np.arange(-90,91,1)

phi = -1*(lat-90)*np.pi/180.0
theta = lon*np.pi/180.0

phi_v, theta_v = np.meshgrid(phi, theta)

#Make face centers
phi_f = 0.5*(phi_v[1:,1:]+phi_v[:-1,:-1])
theta_f = 0.5*(theta_v[1:,1:]+theta_v[:-1,:-1])
lat_f = -1*phi_f*180/np.pi+90
lon_f = theta_f*180/np.pi

x = (r*np.cos(theta_f)*np.sin(phi_f)).flatten()
y = (r*np.sin(theta_f)*np.sin(phi_f)).flatten()
z = (r*np.cos(phi_f)).flatten()
interpolation_spheres_coords_cartesian = np.array([x,y,z])

number_of_points_in_sphere = np.array([])
interpolated_values_array = np.array([])

for i in range(len(interpolation_spheres_coords_cartesian[0])):
    interp_r = np.sqrt(np.sum(interpolation_spheres_coords_cartesian[:,i]**2))
    r_dist = np.abs(dat_r_values - interp_r)
    distance = ((dat_x_values - interpolation_spheres_coords_cartesian[0,i])**2 +
                (dat_y_values - interpolation_spheres_coords_cartesian[1,i])**2 +
                (dat_z_values - interpolation_spheres_coords_cartesian[2,i])**2)
    
    good_points = np.logical_and(distance<=(one_hundred_sixty_km**2),
                                r_dist<=25.0/3390.0)
    good_x, good_y, good_z, good_distances = dat_x_values[good_points], dat_y_values[good_points], dat_z_values[good_points], distance[good_points]
    number_of_points_in_sphere = np.append(number_of_points_in_sphere, len(good_distances))

    dat_H_p1_num_dens_values = f['H_p1_number_density'][:][good_points] #Pulling out values at new indices within interp sphere
    dat_H_p1_v_x = f['H_p1_velocity_x'][:][good_points]
    dat_H_p1_v_y = f['H_p1_velocity_y'][:][good_points]
    dat_H_p1_v_z = f['H_p1_velocity_z'][:][good_points]
    dat_H_p1_v_tot = np.sqrt(dat_H_p1_v_x**2 + dat_H_p1_v_y**2 + dat_H_p1_v_z**2)

    dat_velocity_x = f['velocity_x'][:][good_points] #background fluid velocities
    dat_velocity_y = f['velocity_y'][:][good_points]
    dat_velocity_z = f['velocity_z'][:][good_points]
    dat_velocity_total = np.sqrt(dat_velocity_x**2 + dat_velocity_y**2 + dat_velocity_z**2)
    
    dat_current_x = f['current_x'][:][good_points]
    dat_current_y = f['current_y'][:][good_points]
    dat_current_z = f['current_z'][:][good_points]
    dat_current_total = np.sqrt(dat_current_x**2 + dat_current_y**2 + dat_current_z**2)
    
    #J = np.array([f['current_x'][:][good_points], f['current_y'][:][good_points], f['current_z'][:][good_points]])
    #B = np.array([mag_field_x, mag_field_y, mag_field_z])
    #J_cross_B_x = J[1]*B[2] - J[2]*B[1]
    #J_cross_B_y = J[2]*B[0] - J[0]*B[2]
    #J_cross_B_z = J[0]*B[1] - J[1]*B[0]
    #J_cross_B_tot = np.sqrt(J_cross_B_x**2 + J_cross_B_y**2 + J_cross_B_z**2)
    
    interpolation_fields = np.array([dat_H_p1_num_dens_values, dat_H_p1_v_x, dat_H_p1_v_y, dat_H_p1_v_z, dat_H_p1_v_tot,
                                    dat_velocity_x, dat_velocity_y, dat_velocity_z, dat_velocity_total, dat_current_x,
                                    dat_current_y, dat_current_z, dat_current_total])#J_cross_B_x,J_cross_B_y,J_cross_B_z,J_cross_B_tot

    distances = good_distances #array of distances from center of sphere to points
    weights = 1.0 / (distances) #1/di factor

    for n in range(len(interpolation_fields)):
        numerator = np.sum(weights*interpolation_fields[n])
        denominator = np.sum(weights)
        interpolated_values_array = np.append(interpolated_values_array, (numerator / denominator))

reshaped_interpolated_values_array = np.reshape(interpolated_values_array, (len(interpolation_spheres_coords_cartesian[0]),13))

In [ ]:
#SAVING values to txt file
file = open('LR_800km_interp_vals_v_cross_b.txt','ab') #opens file to append without overwriting
#dataout = (reshaped_interpolated_values_array, interpolation_sphere_coords_spherical))
np.savetxt(file, reshaped_interpolated_values_array, delimiter=',',header=' H+_num_dens, H+_vx, H+_vy, H+_vz, H+_vtot, velocity_x, velocity_y, velocity_z, velocity_total, current_x, current_y, current_z, current_total',
           fmt='%8.10e',comments='#') #saves columnated values as FWHM,xmin,xmax
file.close()

# 900km:

In [31]:
r_sphere_900km = (3390.0+900.0) / 3390.0 #300km above surface
one_hundred_sixty_km = (160.0) / 3390.0
r = r_sphere_900km
lon = np.arange(-90,271,1)
lat = np.arange(-90,91,1)

phi = -1*(lat-90)*np.pi/180.0
theta = lon*np.pi/180.0

phi_v, theta_v = np.meshgrid(phi, theta)

#Make face centers
phi_f = 0.5*(phi_v[1:,1:]+phi_v[:-1,:-1])
theta_f = 0.5*(theta_v[1:,1:]+theta_v[:-1,:-1])
lat_f = -1*phi_f*180/np.pi+90
lon_f = theta_f*180/np.pi

x = (r*np.cos(theta_f)*np.sin(phi_f)).flatten()
y = (r*np.sin(theta_f)*np.sin(phi_f)).flatten()
z = (r*np.cos(phi_f)).flatten()
interpolation_spheres_coords_cartesian = np.array([x,y,z])

In [32]:
number_of_points_in_sphere = np.array([])
interpolated_values_array = np.array([])

for i in range(len(interpolation_spheres_coords_cartesian[0])):
    interp_r = np.sqrt(np.sum(interpolation_spheres_coords_cartesian[:,i]**2))
    r_dist = np.abs(dat_r_values - interp_r)
    distance = ((dat_x_values - interpolation_spheres_coords_cartesian[0,i])**2 +
                (dat_y_values - interpolation_spheres_coords_cartesian[1,i])**2 +
                (dat_z_values - interpolation_spheres_coords_cartesian[2,i])**2)
    
    good_points = np.logical_and(distance<=(one_hundred_sixty_km**2),
                                r_dist<=25.0/3390.0)
    good_x, good_y, good_z, good_distances = dat_x_values[good_points], dat_y_values[good_points], dat_z_values[good_points], distance[good_points]
    number_of_points_in_sphere = np.append(number_of_points_in_sphere, len(good_distances))

    dat_O2_p1_num_dens_values = f['O2_p1_number_density'][:][good_points] #Pulling out values at new indices within interp sphere
    dat_O2_p1_v_x = f['O2_p1_velocity_x'][:][good_points]
    dat_O2_p1_v_y = f['O2_p1_velocity_y'][:][good_points]
    dat_O2_p1_v_z = f['O2_p1_velocity_z'][:][good_points]
    dat_O2_p1_v_tot = np.sqrt(dat_O2_p1_v_x**2 + dat_O2_p1_v_y**2 + dat_O2_p1_v_z**2)

    dat_O_p1_num_dens_values = f['O_p1_number_density'][:][good_points]
    dat_O_p1_v_x = f['O_p1_velocity_x'][:][good_points]
    dat_O_p1_v_y = f['O_p1_velocity_y'][:][good_points]
    dat_O_p1_v_z = f['O_p1_velocity_z'][:][good_points]
    dat_O_p1_v_tot = np.sqrt(dat_O_p1_v_x**2 + dat_O_p1_v_y**2 + dat_O_p1_v_z**2)
    
    mag_field_x = f['magnetic_field_x'][:][good_points]
    mag_field_y = f['magnetic_field_y'][:][good_points]
    mag_field_z = f['magnetic_field_z'][:][good_points]
    mag_field_tot = np.sqrt(mag_field_x**2 + mag_field_y**2 + mag_field_z**2)
    
    p = np.array([good_x, good_y, good_z])
    normal = p/np.sqrt(np.sum(p**2, axis=0))
    norm = np.array([mag_field_x,mag_field_y,mag_field_z])
    mag_field_norm = np.sum(normal*norm, axis=0)
    
    interpolation_fields = np.array([dat_O2_p1_num_dens_values, dat_O2_p1_v_x, dat_O2_p1_v_y, dat_O2_p1_v_z, dat_O2_p1_v_tot,
                             dat_O_p1_num_dens_values, dat_O_p1_v_x, dat_O_p1_v_y, dat_O_p1_v_z, dat_O_p1_v_tot, 
                             mag_field_x, mag_field_y, mag_field_z, mag_field_tot, mag_field_norm])

    distances = good_distances #array of distances from center of sphere to points
    weights = 1.0 / (distances) #1/di factor

    for n in range(len(interpolation_fields)):
        numerator = np.sum(weights*interpolation_fields[n])
        denominator = np.sum(weights)
        interpolated_values_array = np.append(interpolated_values_array, (numerator / denominator))

reshaped_interpolated_values_array = np.reshape(interpolated_values_array, (len(interpolation_spheres_coords_cartesian[0]),15))

In [33]:
print np.min(number_of_points_in_sphere)
print np.max(number_of_points_in_sphere)

1.0
88.0


In [34]:
interpolation_sphere_coords_spherical = conversion_cart_to_sphere(interpolation_spheres_coords_cartesian[0], interpolation_spheres_coords_cartesian[1], interpolation_spheres_coords_cartesian[2])
stack_coords = np.column_stack(interpolation_sphere_coords_spherical)
stacked_interp_vals = np.hstack((reshaped_interpolated_values_array, stack_coords))

#SAVING values to txt file
file = open('LR_900km_interp_vals.txt','ab') #opens file to append without overwriting
#dataout = (reshaped_interpolated_values_array, interpolation_sphere_coords_spherical))
np.savetxt(file, stacked_interp_vals, delimiter=',',header=' O2+_num_dens, O2+_vx, O2+_vy, O2+_vz, O2+_vtot, O+_num_dens, O+_vx, O+_vy, O+_vz, O+_vtot, mag_x, mag_y, mag_z, mag_tot, mag_norm, r, theta, phi',
           fmt='%8.10e',comments='#') #saves columnated values as FWHM,xmin,xmax
file.close()

## V cross B values: 

In [ ]:
r_sphere_900km = (3390.0+900.0) / 3390.0 #300km above surface
one_hundred_sixty_km = (160.0) / 3390.0
r = r_sphere_900km
lon = np.arange(-90,271,1)
lat = np.arange(-90,91,1)

phi = -1*(lat-90)*np.pi/180.0
theta = lon*np.pi/180.0

phi_v, theta_v = np.meshgrid(phi, theta)

#Make face centers
phi_f = 0.5*(phi_v[1:,1:]+phi_v[:-1,:-1])
theta_f = 0.5*(theta_v[1:,1:]+theta_v[:-1,:-1])
lat_f = -1*phi_f*180/np.pi+90
lon_f = theta_f*180/np.pi

x = (r*np.cos(theta_f)*np.sin(phi_f)).flatten()
y = (r*np.sin(theta_f)*np.sin(phi_f)).flatten()
z = (r*np.cos(phi_f)).flatten()
interpolation_spheres_coords_cartesian = np.array([x,y,z])

number_of_points_in_sphere = np.array([])
interpolated_values_array = np.array([])

for i in range(len(interpolation_spheres_coords_cartesian[0])):
    interp_r = np.sqrt(np.sum(interpolation_spheres_coords_cartesian[:,i]**2))
    r_dist = np.abs(dat_r_values - interp_r)
    distance = ((dat_x_values - interpolation_spheres_coords_cartesian[0,i])**2 +
                (dat_y_values - interpolation_spheres_coords_cartesian[1,i])**2 +
                (dat_z_values - interpolation_spheres_coords_cartesian[2,i])**2)
    
    good_points = np.logical_and(distance<=(one_hundred_sixty_km**2),
                                r_dist<=25.0/3390.0)
    good_x, good_y, good_z, good_distances = dat_x_values[good_points], dat_y_values[good_points], dat_z_values[good_points], distance[good_points]
    number_of_points_in_sphere = np.append(number_of_points_in_sphere, len(good_distances))

    dat_H_p1_num_dens_values = f['H_p1_number_density'][:][good_points] #Pulling out values at new indices within interp sphere
    dat_H_p1_v_x = f['H_p1_velocity_x'][:][good_points]
    dat_H_p1_v_y = f['H_p1_velocity_y'][:][good_points]
    dat_H_p1_v_z = f['H_p1_velocity_z'][:][good_points]
    dat_H_p1_v_tot = np.sqrt(dat_H_p1_v_x**2 + dat_H_p1_v_y**2 + dat_H_p1_v_z**2)

    dat_velocity_x = f['velocity_x'][:][good_points] #background fluid velocities
    dat_velocity_y = f['velocity_y'][:][good_points]
    dat_velocity_z = f['velocity_z'][:][good_points]
    dat_velocity_total = np.sqrt(dat_velocity_x**2 + dat_velocity_y**2 + dat_velocity_z**2)
    
    dat_current_x = f['current_x'][:][good_points]
    dat_current_y = f['current_y'][:][good_points]
    dat_current_z = f['current_z'][:][good_points]
    dat_current_total = np.sqrt(dat_current_x**2 + dat_current_y**2 + dat_current_z**2)
    
    #J = np.array([f['current_x'][:][good_points], f['current_y'][:][good_points], f['current_z'][:][good_points]])
    #B = np.array([mag_field_x, mag_field_y, mag_field_z])
    #J_cross_B_x = J[1]*B[2] - J[2]*B[1]
    #J_cross_B_y = J[2]*B[0] - J[0]*B[2]
    #J_cross_B_z = J[0]*B[1] - J[1]*B[0]
    #J_cross_B_tot = np.sqrt(J_cross_B_x**2 + J_cross_B_y**2 + J_cross_B_z**2)
    
    interpolation_fields = np.array([dat_H_p1_num_dens_values, dat_H_p1_v_x, dat_H_p1_v_y, dat_H_p1_v_z, dat_H_p1_v_tot,
                                    dat_velocity_x, dat_velocity_y, dat_velocity_z, dat_velocity_total, dat_current_x,
                                    dat_current_y, dat_current_z, dat_current_total])#J_cross_B_x,J_cross_B_y,J_cross_B_z,J_cross_B_tot

    distances = good_distances #array of distances from center of sphere to points
    weights = 1.0 / (distances) #1/di factor

    for n in range(len(interpolation_fields)):
        numerator = np.sum(weights*interpolation_fields[n])
        denominator = np.sum(weights)
        interpolated_values_array = np.append(interpolated_values_array, (numerator / denominator))

reshaped_interpolated_values_array = np.reshape(interpolated_values_array, (len(interpolation_spheres_coords_cartesian[0]),13))

In [ ]:
#SAVING values to txt file
file = open('LR_900km_interp_vals_v_cross_b.txt','ab') #opens file to append without overwriting
#dataout = (reshaped_interpolated_values_array, interpolation_sphere_coords_spherical))
np.savetxt(file, reshaped_interpolated_values_array, delimiter=',',header=' H+_num_dens, H+_vx, H+_vy, H+_vz, H+_vtot, velocity_x, velocity_y, velocity_z, velocity_total, current_x, current_y, current_z, current_total',
           fmt='%8.10e',comments='#') #saves columnated values as FWHM,xmin,xmax
file.close()

# 1000km:

In [37]:
r_sphere_1000km = (3390.0+1000.0) / 3390.0 
one_hundred_sixty_five_km = (165.0) / 3390.0
r = r_sphere_1000km
lon = np.arange(-90,271,1)
lat = np.arange(-90,91,1)

phi = -1*(lat-90)*np.pi/180.0
theta = lon*np.pi/180.0

phi_v, theta_v = np.meshgrid(phi, theta)

#Make face centers
phi_f = 0.5*(phi_v[1:,1:]+phi_v[:-1,:-1])
theta_f = 0.5*(theta_v[1:,1:]+theta_v[:-1,:-1])
lat_f = -1*phi_f*180/np.pi+90
lon_f = theta_f*180/np.pi

x = (r*np.cos(theta_f)*np.sin(phi_f)).flatten()
y = (r*np.sin(theta_f)*np.sin(phi_f)).flatten()
z = (r*np.cos(phi_f)).flatten()
interpolation_spheres_coords_cartesian = np.array([x,y,z])

In [38]:
number_of_points_in_sphere = np.array([])
interpolated_values_array = np.array([])

for i in range(len(interpolation_spheres_coords_cartesian[0])):
    interp_r = np.sqrt(np.sum(interpolation_spheres_coords_cartesian[:,i]**2))
    r_dist = np.abs(dat_r_values - interp_r)
    distance = ((dat_x_values - interpolation_spheres_coords_cartesian[0,i])**2 +
                (dat_y_values - interpolation_spheres_coords_cartesian[1,i])**2 +
                (dat_z_values - interpolation_spheres_coords_cartesian[2,i])**2)
    
    good_points = np.logical_and(distance<=(one_hundred_sixty_five_km**2),
                                r_dist<=25.0/3390.0)
    good_x, good_y, good_z, good_distances = dat_x_values[good_points], dat_y_values[good_points], dat_z_values[good_points], distance[good_points]
    number_of_points_in_sphere = np.append(number_of_points_in_sphere, len(good_distances))

    dat_O2_p1_num_dens_values = f['O2_p1_number_density'][:][good_points] #Pulling out values at new indices within interp sphere
    dat_O2_p1_v_x = f['O2_p1_velocity_x'][:][good_points]
    dat_O2_p1_v_y = f['O2_p1_velocity_y'][:][good_points]
    dat_O2_p1_v_z = f['O2_p1_velocity_z'][:][good_points]
    dat_O2_p1_v_tot = np.sqrt(dat_O2_p1_v_x**2 + dat_O2_p1_v_y**2 + dat_O2_p1_v_z**2)

    dat_O_p1_num_dens_values = f['O_p1_number_density'][:][good_points]
    dat_O_p1_v_x = f['O_p1_velocity_x'][:][good_points]
    dat_O_p1_v_y = f['O_p1_velocity_y'][:][good_points]
    dat_O_p1_v_z = f['O_p1_velocity_z'][:][good_points]
    dat_O_p1_v_tot = np.sqrt(dat_O_p1_v_x**2 + dat_O_p1_v_y**2 + dat_O_p1_v_z**2)
    
    mag_field_x = f['magnetic_field_x'][:][good_points]
    mag_field_y = f['magnetic_field_y'][:][good_points]
    mag_field_z = f['magnetic_field_z'][:][good_points]
    mag_field_tot = np.sqrt(mag_field_x**2 + mag_field_y**2 + mag_field_z**2)
    
    p = np.array([good_x, good_y, good_z])
    normal = p/np.sqrt(np.sum(p**2, axis=0))
    norm = np.array([mag_field_x,mag_field_y,mag_field_z])
    mag_field_norm = np.sum(normal*norm, axis=0)
    
    interpolation_fields = np.array([dat_O2_p1_num_dens_values, dat_O2_p1_v_x, dat_O2_p1_v_y, dat_O2_p1_v_z, dat_O2_p1_v_tot,
                             dat_O_p1_num_dens_values, dat_O_p1_v_x, dat_O_p1_v_y, dat_O_p1_v_z, dat_O_p1_v_tot, 
                             mag_field_x, mag_field_y, mag_field_z, mag_field_tot, mag_field_norm])

    distances = good_distances #array of distances from center of sphere to points
    weights = 1.0 / (distances) #1/di factor

    for n in range(len(interpolation_fields)):
        numerator = np.sum(weights*interpolation_fields[n])
        denominator = np.sum(weights)
        interpolated_values_array = np.append(interpolated_values_array, (numerator / denominator))

reshaped_interpolated_values_array = np.reshape(interpolated_values_array, (len(interpolation_spheres_coords_cartesian[0]),15))

In [39]:
print np.min(number_of_points_in_sphere)
print np.max(number_of_points_in_sphere)

2.0
176.0


In [40]:
interpolation_sphere_coords_spherical = conversion_cart_to_sphere(interpolation_spheres_coords_cartesian[0], interpolation_spheres_coords_cartesian[1], interpolation_spheres_coords_cartesian[2])
stack_coords = np.column_stack(interpolation_sphere_coords_spherical)
stacked_interp_vals = np.hstack((reshaped_interpolated_values_array, stack_coords))

#SAVING values to txt file
file = open('LR_1000km_interp_vals.txt','ab') #opens file to append without overwriting
#dataout = (reshaped_interpolated_values_array, interpolation_sphere_coords_spherical))
np.savetxt(file, stacked_interp_vals, delimiter=',',header=' O2+_num_dens, O2+_vx, O2+_vy, O2+_vz, O2+_vtot, O+_num_dens, O+_vx, O+_vy, O+_vz, O+_vtot, mag_x, mag_y, mag_z, mag_tot, mag_norm, r, theta, phi',
           fmt='%8.10e',comments='#') #saves columnated values as FWHM,xmin,xmax
file.close()

## V cross B values: 

In [ ]:
r_sphere_1000km = (3390.0+1000.0) / 3390.0 
one_hundred_sixty_five_km = (165.0) / 3390.0
r = r_sphere_1000km
lon = np.arange(-90,271,1)
lat = np.arange(-90,91,1)

phi = -1*(lat-90)*np.pi/180.0
theta = lon*np.pi/180.0

phi_v, theta_v = np.meshgrid(phi, theta)

#Make face centers
phi_f = 0.5*(phi_v[1:,1:]+phi_v[:-1,:-1])
theta_f = 0.5*(theta_v[1:,1:]+theta_v[:-1,:-1])
lat_f = -1*phi_f*180/np.pi+90
lon_f = theta_f*180/np.pi

x = (r*np.cos(theta_f)*np.sin(phi_f)).flatten()
y = (r*np.sin(theta_f)*np.sin(phi_f)).flatten()
z = (r*np.cos(phi_f)).flatten()
interpolation_spheres_coords_cartesian = np.array([x,y,z])

number_of_points_in_sphere = np.array([])
interpolated_values_array = np.array([])

for i in range(len(interpolation_spheres_coords_cartesian[0])):
    interp_r = np.sqrt(np.sum(interpolation_spheres_coords_cartesian[:,i]**2))
    r_dist = np.abs(dat_r_values - interp_r)
    distance = ((dat_x_values - interpolation_spheres_coords_cartesian[0,i])**2 +
                (dat_y_values - interpolation_spheres_coords_cartesian[1,i])**2 +
                (dat_z_values - interpolation_spheres_coords_cartesian[2,i])**2)
    
    good_points = np.logical_and(distance<=(one_hundred_sixty_five_km**2),
                                r_dist<=25.0/3390.0)
    good_x, good_y, good_z, good_distances = dat_x_values[good_points], dat_y_values[good_points], dat_z_values[good_points], distance[good_points]
    number_of_points_in_sphere = np.append(number_of_points_in_sphere, len(good_distances))

    dat_H_p1_num_dens_values = f['H_p1_number_density'][:][good_points] #Pulling out values at new indices within interp sphere
    dat_H_p1_v_x = f['H_p1_velocity_x'][:][good_points]
    dat_H_p1_v_y = f['H_p1_velocity_y'][:][good_points]
    dat_H_p1_v_z = f['H_p1_velocity_z'][:][good_points]
    dat_H_p1_v_tot = np.sqrt(dat_H_p1_v_x**2 + dat_H_p1_v_y**2 + dat_H_p1_v_z**2)

    dat_velocity_x = f['velocity_x'][:][good_points] #background fluid velocities
    dat_velocity_y = f['velocity_y'][:][good_points]
    dat_velocity_z = f['velocity_z'][:][good_points]
    dat_velocity_total = np.sqrt(dat_velocity_x**2 + dat_velocity_y**2 + dat_velocity_z**2)
    
    dat_current_x = f['current_x'][:][good_points]
    dat_current_y = f['current_y'][:][good_points]
    dat_current_z = f['current_z'][:][good_points]
    dat_current_total = np.sqrt(dat_current_x**2 + dat_current_y**2 + dat_current_z**2)
    
    #J = np.array([f['current_x'][:][good_points], f['current_y'][:][good_points], f['current_z'][:][good_points]])
    #B = np.array([mag_field_x, mag_field_y, mag_field_z])
    #J_cross_B_x = J[1]*B[2] - J[2]*B[1]
    #J_cross_B_y = J[2]*B[0] - J[0]*B[2]
    #J_cross_B_z = J[0]*B[1] - J[1]*B[0]
    #J_cross_B_tot = np.sqrt(J_cross_B_x**2 + J_cross_B_y**2 + J_cross_B_z**2)
    
    interpolation_fields = np.array([dat_H_p1_num_dens_values, dat_H_p1_v_x, dat_H_p1_v_y, dat_H_p1_v_z, dat_H_p1_v_tot,
                                    dat_velocity_x, dat_velocity_y, dat_velocity_z, dat_velocity_total, dat_current_x,
                                    dat_current_y, dat_current_z, dat_current_total])#J_cross_B_x,J_cross_B_y,J_cross_B_z,J_cross_B_tot

    distances = good_distances #array of distances from center of sphere to points
    weights = 1.0 / (distances) #1/di factor

    for n in range(len(interpolation_fields)):
        numerator = np.sum(weights*interpolation_fields[n])
        denominator = np.sum(weights)
        interpolated_values_array = np.append(interpolated_values_array, (numerator / denominator))

reshaped_interpolated_values_array = np.reshape(interpolated_values_array, (len(interpolation_spheres_coords_cartesian[0]),13))

In [ ]:
#SAVING values to txt file
file = open('LR_1000km_interp_vals_v_cross_b.txt','ab') #opens file to append without overwriting
#dataout = (reshaped_interpolated_values_array, interpolation_sphere_coords_spherical))
np.savetxt(file, reshaped_interpolated_values_array, delimiter=',',header=' H+_num_dens, H+_vx, H+_vy, H+_vz, H+_vtot, velocity_x, velocity_y, velocity_z, velocity_total, current_x, current_y, current_z, current_total',
           fmt='%8.10e',comments='#') #saves columnated values as FWHM,xmin,xmax
file.close()